<a href="https://colab.research.google.com/github/leosammallahti/AnalysisCoLab/blob/main/QualityImprovement1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Setup and Load Data
import pandas as pd
import numpy as np
from google.colab import drive

# Mount drive
drive.mount('/content/drive')

# Load the data
df = pd.read_csv('/content/drive/MyDrive/AEA_RCT_Parsed/structured_studies_full_v2.csv')

print(f"Total rows: {len(df)}")
print(f"Rows with empty abstract: {df['abstract'].isna().sum()}")
print(f"Rows with populated abstract: {df['abstract'].notna().sum()}")

Mounted at /content/drive
Total rows: 2289
Rows with empty abstract: 158
Rows with populated abstract: 2131


In [ ]:
# Cell 2: Identify Abstract Recovery Candidates
# Find rows where abstract is empty/null
empty_abstract_mask = df['abstract'].isna() | (df['abstract'].str.strip() == '')
df_empty_abstract = df[empty_abstract_mask].copy()

print(f"Found {len(df_empty_abstract)} rows with empty/null abstract")

# Define the relevant abstract columns to check
relevant_abstract_cols = [
    'relevant_1_abstract', 'relevant_2_abstract', 'relevant_3_abstract',
    'relevant_4_abstract', 'relevant_5_abstract', 'relevant_6_abstract',
    'relevant_7_abstract'
]

# Check which of these columns exist in the dataframe
existing_relevant_cols = [col for col in relevant_abstract_cols if col in df.columns]
print(f"\nFound {len(existing_relevant_cols)} relevant_abstract columns in dataset")

Found 158 rows with empty/null abstract

Found 7 relevant_abstract columns in dataset


In [ ]:
# Cell 3: Analyze Recovery Potential
recovery_candidates = []

for idx, row in df_empty_abstract.iterrows():
    # Check each relevant_abstract column for this row
    for col in existing_relevant_cols:
        if pd.notna(row[col]) and str(row[col]).strip() != '':
            recovery_candidates.append({
                'rct_id': row['rct_id'],
                'title': row['title'][:100] if pd.notna(row['title']) else 'NO TITLE',
                'source_column': col,
                'abstract_preview': str(row[col])[:200] + '...' if len(str(row[col])) > 200 else str(row[col]),
                'abstract_length': len(str(row[col]))
            })

recovery_df = pd.DataFrame(recovery_candidates)

print(f"Found {len(recovery_df)} potential abstract recovery cases")
print(f"Unique RCTs that could be recovered: {recovery_df['rct_id'].nunique()}")

Found 0 potential abstract recovery cases


KeyError: 'rct_id'

In [ ]:
# Cell 3: Analyze Recovery Potential (Fixed)
recovery_candidates = []

for idx, row in df_empty_abstract.iterrows():
    # Check each relevant_abstract column for this row
    for col in existing_relevant_cols:
        if pd.notna(row[col]) and str(row[col]).strip() != '':
            recovery_candidates.append({
                'rct_id': row['rct_id'],
                'title': row['title'][:100] if pd.notna(row['title']) else 'NO TITLE',
                'source_column': col,
                'abstract_preview': str(row[col])[:200] + '...' if len(str(row[col])) > 200 else str(row[col]),
                'abstract_length': len(str(row[col]))
            })

recovery_df = pd.DataFrame(recovery_candidates)

print(f"Found {len(recovery_df)} potential abstract recovery cases")

if len(recovery_df) > 0:
    print(f"Unique RCTs that could be recovered: {recovery_df['rct_id'].nunique()}")
else:
    print("\nNo recovery candidates found. Investigating further...")

    # Diagnostic: Check if relevant columns have ANY data at all
    print("\nChecking relevant_abstract columns overall population:")
    for col in existing_relevant_cols:
        populated = df[col].notna().sum()
        print(f"  {col}: {populated} populated rows (out of {len(df)} total)")

    # Check if these columns have data but NOT for the empty abstract rows
    print("\nChecking if empty abstract rows have ANY relevant_abstract data:")
    for col in existing_relevant_cols:
        if col in df_empty_abstract.columns:
            populated_in_empty = df_empty_abstract[col].notna().sum()
            print(f"  {col}: {populated_in_empty} populated (out of {len(df_empty_abstract)} empty abstract rows)")

Found 0 potential abstract recovery cases

No recovery candidates found. Investigating further...

Checking relevant_abstract columns overall population:
  relevant_1_abstract: 645 populated rows (out of 2289 total)
  relevant_2_abstract: 94 populated rows (out of 2289 total)
  relevant_3_abstract: 18 populated rows (out of 2289 total)
  relevant_4_abstract: 5 populated rows (out of 2289 total)
  relevant_5_abstract: 3 populated rows (out of 2289 total)
  relevant_6_abstract: 2 populated rows (out of 2289 total)
  relevant_7_abstract: 0 populated rows (out of 2289 total)

Checking if empty abstract rows have ANY relevant_abstract data:
  relevant_1_abstract: 0 populated (out of 158 empty abstract rows)
  relevant_2_abstract: 0 populated (out of 158 empty abstract rows)
  relevant_3_abstract: 0 populated (out of 158 empty abstract rows)
  relevant_4_abstract: 0 populated (out of 158 empty abstract rows)
  relevant_5_abstract: 0 populated (out of 158 empty abstract rows)
  relevant_6_abs

In [ ]:
# Cell 5: Analyze Field Availability and Quality for New CSV
print("FIELD MAPPING ANALYSIS FOR NEW CONSOLIDATED CSV")
print("="*80)

# Check what we have vs what we need
field_mapping = {
    'Title': 'title',
    'Researcher(s)': ['primary_investigator_name', 'other_pi_1_name', 'other_pi_2_name', 'other_pi_3_name', 'other_pi_4_name'],
    'Researcher affiliation': ['primary_investigator_affiliation', 'other_pi_1_affiliation', 'other_pi_2_affiliation', 'other_pi_3_affiliation', 'other_pi_4_affiliation'],
    'Abstract': 'abstract',
    'Description of the intervention': 'interventions',
    'Primary outcomes': 'primary_outcomes_endpoints',
    'Secondary outcomes': 'secondary_outcomes_endpoints',
    'Findings': ['final_findings_snippet', 'findings_snippet', 'llm_snippet', 'findings_fulltext'],
    'Country': 'country',
    'Subregion': 'region',
    'Year': ['start_date', 'first_published', 'initial_registration_date'],
    'Population': 'population_extracted',  # From AI enrichment
    'Search vector': 'search_vector',
    'keywords': 'keywords',
    'keywords_additional': 'keywords_additional',
    'jel_codes': 'jel_codes'
}

# Check availability and population rates
for target, source in field_mapping.items():
    if isinstance(source, list):
        print(f"\n{target} (needs consolidation from multiple fields):")
        for s in source:
            if s in df.columns:
                populated = df[s].notna().sum()
                print(f"  - {s}: {populated}/{len(df)} ({populated/len(df)*100:.1f}%)")
            else:
                print(f"  - {s}: NOT IN DATASET")
    else:
        if source in df.columns:
            populated = df[source].notna().sum()
            print(f"\n{target}: {source} - {populated}/{len(df)} ({populated/len(df)*100:.1f}%)")
        else:
            print(f"\n{target}: {source} - NOT IN DATASET")

# Check for AI-generated keyword fields that might not exist yet
print("\n" + "="*80)
print("CHECKING FOR KEYWORD SUBCATEGORIES:")
keyword_fields = ['keywords_sector', 'keywords_mechanisms', 'keywords_implementation', 'keywords_context']
for field in keyword_fields:
    if field in df.columns:
        populated = df[field].notna().sum()
        print(f"  {field}: {populated}/{len(df)} ({populated/len(df)*100:.1f}%)")
    else:
        print(f"  {field}: NOT IN DATASET - needs to be generated")

# Check for duplicates
print("\n" + "="*80)
print("DUPLICATE ANALYSIS:")
print(f"Total rows: {len(df)}")
print(f"Unique rct_ids: {df['rct_id'].nunique()}")
print(f"Duplicate rct_ids: {df['rct_id'].duplicated().sum()}")

if df['rct_id'].duplicated().any():
    print("\nDuplicate RCT IDs found:")
    duplicates = df[df['rct_id'].duplicated(keep=False)].sort_values('rct_id')
    print(duplicates[['rct_id', 'title']].head(10))

# Check title duplicates (different studies with same title)
title_duplicates = df[df['title'].duplicated(keep=False) & df['title'].notna()]
if len(title_duplicates) > 0:
    print(f"\nRows with duplicate titles: {len(title_duplicates)}")
    print("Sample of duplicate titles:")
    for title in title_duplicates['title'].unique()[:3]:
        matching = df[df['title'] == title][['rct_id', 'title', 'primary_investigator_name']]
        print(f"\n  Title: '{title[:80]}...'")
        print(f"  Found in {len(matching)} rows")

FIELD MAPPING ANALYSIS FOR NEW CONSOLIDATED CSV

Title: title - 2286/2289 (99.9%)

Researcher(s) (needs consolidation from multiple fields):
  - primary_investigator_name: 2288/2289 (100.0%)
  - other_pi_1_name: 1720/2289 (75.1%)
  - other_pi_2_name: 14/2289 (0.6%)
  - other_pi_3_name: 4/2289 (0.2%)
  - other_pi_4_name: 1/2289 (0.0%)

Researcher affiliation (needs consolidation from multiple fields):
  - primary_investigator_affiliation: 2136/2289 (93.3%)
  - other_pi_1_affiliation: 468/2289 (20.4%)
  - other_pi_2_affiliation: 1/2289 (0.0%)
  - other_pi_3_affiliation: 2/2289 (0.1%)
  - other_pi_4_affiliation: 0/2289 (0.0%)

Abstract: abstract - 2131/2289 (93.1%)

Description of the intervention: interventions - 2136/2289 (93.3%)

Primary outcomes: primary_outcomes_endpoints - 2136/2289 (93.3%)

Secondary outcomes: secondary_outcomes_endpoints - 2136/2289 (93.3%)

Findings (needs consolidation from multiple fields):
  - final_findings_snippet: 2289/2289 (100.0%)
  - findings_snippet: 76

In [ ]:
# Cell 6: Investigate Duplicate RCT IDs in Detail
print("DETAILED DUPLICATE ANALYSIS")
print("="*80)

# Get all duplicate rct_ids
duplicate_mask = df['rct_id'].duplicated(keep=False)
df_duplicates = df[duplicate_mask].sort_values('rct_id')

print(f"Found {len(df_duplicates)} rows with duplicate rct_ids")
print(f"Unique duplicate rct_ids: {df_duplicates['rct_id'].nunique()}")

# Analyze each duplicate group
duplicate_groups = df_duplicates.groupby('rct_id')

for rct_id, group in duplicate_groups:
    print(f"\n{'='*80}")
    print(f"RCT ID: {rct_id} - Found in {len(group)} rows")
    print("-"*80)

    # Check key fields for differences
    fields_to_compare = [
        'title', 'primary_investigator_name', 'final_has_findings',
        'final_findings_source_type', 'start_date', 'country'
    ]

    for field in fields_to_compare:
        if field in group.columns:
            unique_values = group[field].unique()
            if len(unique_values) > 1:
                print(f"  {field}: DIFFERENT VALUES")
                for idx, row in group.iterrows():
                    print(f"    Row {idx}: {str(row[field])[:100]}")
            else:
                print(f"  {field}: {unique_values[0] if pd.notna(unique_values[0]) else 'NaN'}")

    # Check which has findings
    print("\n  Findings status:")
    for idx, row in group.iterrows():
        has_findings = row.get('final_has_findings', 'N/A')
        snippet_length = len(str(row.get('final_findings_snippet', ''))) if pd.notna(row.get('final_findings_snippet')) else 0
        print(f"    Row {idx}: has_findings={has_findings}, snippet_length={snippet_length}")

# Check if duplicates come from different source files (if that column exists)
if 'source_file' in df.columns:
    print("\n" + "="*80)
    print("SOURCE FILE ANALYSIS FOR DUPLICATES:")
    for rct_id, group in duplicate_groups:
        sources = group['source_file'].unique()
        print(f"  RCT {rct_id}: from files {sources}")

# Recommendation for handling duplicates
print("\n" + "="*80)
print("DUPLICATE RESOLUTION STRATEGY:")
print("1. Check if duplicates are from WITH_findings vs WITHOUT_findings merges")
print("2. Keep the row with the most complete data (especially findings)")
print("3. Create a deduplication log showing which row was kept and why")

DETAILED DUPLICATE ANALYSIS
Found 14 rows with duplicate rct_ids
Unique duplicate rct_ids: 7

RCT ID: 2868.0 - Found in 2 rows
--------------------------------------------------------------------------------
  title: Experimental evaluation of Semillas de Apego, a group-based program to foster maternal mental health and early childhood development among violence exposed communities in Colombia.
  primary_investigator_name: Andres Moya
  final_has_findings: Yes
  final_findings_source_type: DIFFERENT VALUES
    Row 124: institutional
    Row 1752: unknown
  start_date: DIFFERENT VALUES
    Row 124: 2018-08-19
    Row 1752: nan
  country: DIFFERENT VALUES
    Row 124: Colombia
    Row 1752: nan

  Findings status:
    Row 124: has_findings=Yes, snippet_length=357
    Row 1752: has_findings=Yes, snippet_length=361

RCT ID: 3283.0 - Found in 2 rows
--------------------------------------------------------------------------------
  title: Networks and Global Health: Experimental Evidence of 

In [ ]:
# Cell 7: Deduplicate by Keeping Most Complete Row
print("DEDUPLICATION PROCESS")
print("="*80)

# Create a copy for deduplication
df_clean = df.copy()

# Function to score row completeness
def calculate_completeness_score(row):
    """Score based on important fields being populated"""
    score = 0
    important_fields = {
        'start_date': 10,  # High weight for metadata
        'country': 10,
        'abstract': 5,
        'interventions': 5,
        'primary_outcomes_endpoints': 3,
        'region': 3,
        'final_findings_snippet': 2,
    }

    for field, weight in important_fields.items():
        if field in row.index and pd.notna(row[field]) and str(row[field]).strip() != '':
            score += weight
            # Bonus for longer content in text fields
            if field in ['abstract', 'final_findings_snippet', 'interventions']:
                score += min(len(str(row[field])) / 100, 5)  # Up to 5 bonus points

    return score

# Process duplicates
dedup_log = []
rows_to_drop = []

for rct_id in df_clean[df_clean['rct_id'].duplicated(keep=False)]['rct_id'].unique():
    duplicate_rows = df_clean[df_clean['rct_id'] == rct_id]

    # Score each duplicate
    scores = {}
    for idx in duplicate_rows.index:
        scores[idx] = calculate_completeness_score(duplicate_rows.loc[idx])

    # Keep the row with highest score
    best_idx = max(scores, key=scores.get)
    drop_indices = [idx for idx in scores.keys() if idx != best_idx]

    # Log the decision
    dedup_log.append({
        'rct_id': rct_id,
        'kept_row': best_idx,
        'kept_score': scores[best_idx],
        'dropped_rows': drop_indices,
        'dropped_scores': [scores[idx] for idx in drop_indices],
        'title': df_clean.loc[best_idx, 'title'] if pd.notna(df_clean.loc[best_idx, 'title']) else 'NO TITLE'
    })

    rows_to_drop.extend(drop_indices)

# Drop duplicate rows
df_clean = df_clean.drop(index=rows_to_drop)

# Report results
print(f"Original rows: {len(df)}")
print(f"Rows removed: {len(rows_to_drop)}")
print(f"Final rows: {len(df_clean)}")
print(f"Unique rct_ids: {df_clean['rct_id'].nunique()}")

print("\n" + "="*80)
print("DEDUPLICATION LOG:")
for entry in dedup_log:
    print(f"\nRCT {entry['rct_id']}: {entry['title'][:60]}...")
    print(f"  Kept row {entry['kept_row']} (score: {entry['kept_score']:.1f})")
    print(f"  Dropped row(s) {entry['dropped_rows']} (scores: {entry['dropped_scores']})")

# Verify no more duplicates
if df_clean['rct_id'].duplicated().any():
    print("\nWARNING: Still have duplicates!")
else:
    print("\nSUCCESS: No more duplicate rct_ids")

# Update df to use the clean version
df = df_clean
print(f"\nDataset updated. Working with {len(df)} unique studies.")

DEDUPLICATION PROCESS
Original rows: 2289
Rows removed: 7
Final rows: 2282
Unique rct_ids: 2281

DEDUPLICATION LOG:

RCT 2868.0: Experimental evaluation of Semillas de Apego, a group-based ...
  Kept row 124 (score: 51.6)
  Dropped row(s) [1752] (scores: [5.609999999999999])

RCT 3283.0: Networks and Global Health: Experimental Evidence of Women’s...
  Kept row 179 (score: 52.1)
  Dropped row(s) [1808] (scores: [4.76])

RCT 3344.0: Bangladesh Chars Tobacco Assessment Project (CTAP) 2018...
  Kept row 187 (score: 51.0)
  Dropped row(s) [1817] (scores: [6.1])

RCT 5735.0: The Behavioral Effect of Facial Masks During the COVID-19 Pa...
  Kept row 448 (score: 53.0)
  Dropped row(s) [2021] (scores: [5.41])

RCT 5781.0: NO TITLE...
  Kept row 453 (score: 46.6)
  Dropped row(s) [2024] (scores: [3.7199999999999998])

RCT 7650.0: Keep your eyes on the prize: How to make penalty contracts w...
  Kept row 652 (score: 50.0)
  Dropped row(s) [2139] (scores: [5.09])

RCT 7741.0: Non-Linear Public Go

In [ ]:
# Cell 8: Build Consolidated Dataset with Required Fields
print("BUILDING CONSOLIDATED DATASET")
print("="*80)

# Initialize new dataframe with required columns
consolidated = pd.DataFrame()

# 1. Simple direct mappings
consolidated['Title'] = df['title']
consolidated['Country'] = df['country']
consolidated['Subregion'] = df['region']
consolidated['Abstract'] = df['abstract']
consolidated['Description_of_intervention'] = df['interventions']
consolidated['Primary_outcomes'] = df['primary_outcomes_endpoints']
consolidated['Secondary_outcomes'] = df['secondary_outcomes_endpoints']
consolidated['Population'] = df.get('population_extracted', pd.Series([None]*len(df)))
consolidated['Search_vector'] = df['search_vector']
consolidated['keywords'] = df['keywords']
consolidated['keywords_additional'] = df['keywords_additional']
consolidated['jel_codes'] = df['jel_codes']

# Keyword subcategories
consolidated['keywords_sector'] = df.get('keywords_sector', pd.Series([None]*len(df)))
consolidated['keywords_mechanisms'] = df.get('keywords_mechanisms', pd.Series([None]*len(df)))
consolidated['keywords_implementation'] = df.get('keywords_implementation', pd.Series([None]*len(df)))
consolidated['keywords_context'] = df.get('keywords_context', pd.Series([None]*len(df)))

# 2. Consolidated Researchers (combine all PI names)
def consolidate_researchers(row):
    researchers = []
    pi_cols = ['primary_investigator_name', 'other_pi_1_name', 'other_pi_2_name', 'other_pi_3_name', 'other_pi_4_name']
    for col in pi_cols:
        if col in row.index and pd.notna(row[col]) and str(row[col]).strip():
            researchers.append(str(row[col]).strip())
    return '; '.join(researchers) if researchers else None

consolidated['Researchers'] = df.apply(consolidate_researchers, axis=1)

# 3. Consolidated Affiliations
def consolidate_affiliations(row):
    affiliations = []
    aff_cols = ['primary_investigator_affiliation', 'other_pi_1_affiliation', 'other_pi_2_affiliation', 'other_pi_3_affiliation', 'other_pi_4_affiliation']
    for col in aff_cols:
        if col in row.index and pd.notna(row[col]) and str(row[col]).strip():
            affiliations.append(str(row[col]).strip())
    return '; '.join(affiliations) if affiliations else None

consolidated['Researcher_affiliation'] = df.apply(consolidate_affiliations, axis=1)

# 4. Extract Year from dates (prioritize start_date, then first_published, then initial_registration_date)
def extract_year(row):
    for date_col in ['start_date', 'first_published', 'initial_registration_date']:
        if date_col in row.index and pd.notna(row[date_col]):
            try:
                # Handle different date formats
                date_str = str(row[date_col])
                if len(date_str) >= 4:
                    return date_str[:4]
            except:
                continue
    return None

consolidated['Year'] = df.apply(extract_year, axis=1)

# 5. Consolidated Findings (prioritize final_findings_snippet as it's 100% populated)
consolidated['Findings'] = df['final_findings_snippet']

# Add RCT ID for reference
consolidated['rct_id'] = df['rct_id']

# Report on consolidated dataset
print(f"Created consolidated dataset with {len(consolidated)} rows and {len(consolidated.columns)} columns")
print("\nColumn population rates:")
for col in consolidated.columns:
    populated = consolidated[col].notna().sum()
    pct = (populated / len(consolidated)) * 100
    print(f"  {col}: {populated}/{len(consolidated)} ({pct:.1f}%)")

# Check for any remaining issues
print("\n" + "="*80)
print("QUALITY CHECKS:")
print(f"Unique rct_ids: {consolidated['rct_id'].nunique()}")
print(f"Rows with Title: {consolidated['Title'].notna().sum()}")
print(f"Rows with Researchers: {consolidated['Researchers'].notna().sum()}")
print(f"Rows with Abstract: {consolidated['Abstract'].notna().sum()}")
print(f"Rows with Findings: {consolidated['Findings'].notna().sum()}")

BUILDING CONSOLIDATED DATASET
Created consolidated dataset with 2282 rows and 21 columns

Column population rates:
  Title: 2280/2282 (99.9%)
  Country: 2134/2282 (93.5%)
  Subregion: 1002/2282 (43.9%)
  Abstract: 2131/2282 (93.4%)
  Description_of_intervention: 2136/2282 (93.6%)
  Primary_outcomes: 2136/2282 (93.6%)
  Secondary_outcomes: 2136/2282 (93.6%)
  Population: 2110/2282 (92.5%)
  Search_vector: 2282/2282 (100.0%)
  keywords: 2136/2282 (93.6%)
  keywords_additional: 2136/2282 (93.6%)
  jel_codes: 1229/2282 (53.9%)
  keywords_sector: 2116/2282 (92.7%)
  keywords_mechanisms: 2116/2282 (92.7%)
  keywords_implementation: 2117/2282 (92.8%)
  keywords_context: 2114/2282 (92.6%)
  Researchers: 2281/2282 (100.0%)
  Researcher_affiliation: 2136/2282 (93.6%)
  Year: 2136/2282 (93.6%)
  Findings: 2282/2282 (100.0%)
  rct_id: 2281/2282 (100.0%)

QUALITY CHECKS:
Unique rct_ids: 2281
Rows with Title: 2280
Rows with Researchers: 2281
Rows with Abstract: 2131
Rows with Findings: 2282


In [ ]:
# Cell 9: Analyze Missing Data Patterns
print("MISSING DATA PATTERN ANALYSIS")
print("="*80)

# Define key fields to analyze
key_fields = [
    'Title', 'Abstract', 'Country', 'Subregion',
    'Description_of_intervention', 'Primary_outcomes',
    'Secondary_outcomes', 'Population', 'Year',
    'Researcher_affiliation', 'jel_codes'
]

# Create a missing data matrix
missing_matrix = pd.DataFrame()
for field in key_fields:
    missing_matrix[field] = consolidated[field].isna().astype(int)

# Calculate missing field count per study
consolidated['missing_count'] = missing_matrix.sum(axis=1)

# Distribution of missing fields
print("Distribution of missing fields per study:")
missing_distribution = consolidated['missing_count'].value_counts().sort_index()
for num_missing, count in missing_distribution.items():
    pct = (count / len(consolidated)) * 100
    print(f"  {num_missing} fields missing: {count} studies ({pct:.1f}%)")

# Co-occurrence analysis
print("\n" + "="*80)
print("CO-OCCURRENCE OF MISSING DATA:")
print("(If field X is missing, probability that field Y is also missing)\n")

# Calculate conditional probabilities
for field1 in ['Abstract', 'Description_of_intervention', 'Country', 'Year']:
    missing_field1 = consolidated[field1].isna()
    n_missing = missing_field1.sum()

    if n_missing > 0:
        print(f"\nWhen {field1} is missing ({n_missing} cases):")
        for field2 in key_fields:
            if field2 != field1:
                both_missing = (missing_field1 & consolidated[field2].isna()).sum()
                prob = (both_missing / n_missing) * 100
                if prob > 50:  # Only show high correlations
                    print(f"  {field2}: {prob:.1f}% also missing")

# Identify studies with most missing data
print("\n" + "="*80)
print("STUDIES WITH MOST MISSING FIELDS:")
worst_cases = consolidated[consolidated['missing_count'] >= 3].sort_values('missing_count', ascending=False)
print(f"\nFound {len(worst_cases)} studies missing 3+ fields:")

for idx, row in worst_cases.head(10).iterrows():
    missing_fields = [f for f in key_fields if pd.isna(row[f])]
    print(f"\nRCT {row['rct_id']}: {row['Title'][:50] if pd.notna(row['Title']) else 'NO TITLE'}")
    print(f"  Missing {row['missing_count']} fields: {', '.join(missing_fields)}")

# Pattern identification
print("\n" + "="*80)
print("MISSING DATA PATTERNS:")

# Check if there's a "core group" that's always missing together
always_together = []
for i, field1 in enumerate(key_fields):
    for field2 in key_fields[i+1:]:
        missing1 = consolidated[field1].isna()
        missing2 = consolidated[field2].isna()

        # Check if they're almost always missing together
        both_missing = (missing1 & missing2).sum()
        either_missing = (missing1 | missing2).sum()

        if either_missing > 0:
            overlap_ratio = both_missing / either_missing
            if overlap_ratio > 0.9 and both_missing > 100:  # High overlap and significant count
                always_together.append((field1, field2, overlap_ratio, both_missing))

if always_together:
    print("\nFields that are almost always missing together:")
    for f1, f2, ratio, count in always_together:
        print(f"  {f1} + {f2}: {ratio:.1%} overlap ({count} cases)")
else:
    print("\nNo strong patterns of fields always missing together")

# Summary
print("\n" + "="*80)
print("KEY INSIGHTS:")
complete_studies = len(consolidated[consolidated['missing_count'] == 0])
print(f"- {complete_studies}/{len(consolidated)} ({complete_studies/len(consolidated)*100:.1f}%) studies have ALL key fields")
print(f"- {len(worst_cases)} studies ({len(worst_cases)/len(consolidated)*100:.1f}%) are missing 3+ fields")
print(f"- Subregion and JEL codes are most commonly missing across all studies")

MISSING DATA PATTERN ANALYSIS
Distribution of missing fields per study:
  0 fields missing: 631 studies (27.7%)
  1 fields missing: 955 studies (41.8%)
  2 fields missing: 532 studies (23.3%)
  3 fields missing: 16 studies (0.7%)
  4 fields missing: 2 studies (0.1%)
  10 fields missing: 145 studies (6.4%)
  11 fields missing: 1 studies (0.0%)

CO-OCCURRENCE OF MISSING DATA:
(If field X is missing, probability that field Y is also missing)


When Abstract is missing (151 cases):
  Country: 96.7% also missing
  Subregion: 97.4% also missing
  Description_of_intervention: 96.7% also missing
  Primary_outcomes: 96.7% also missing
  Secondary_outcomes: 96.7% also missing
  Population: 97.4% also missing
  Year: 96.7% also missing
  Researcher_affiliation: 96.7% also missing
  jel_codes: 99.3% also missing

When Description_of_intervention is missing (146 cases):
  Abstract: 100.0% also missing
  Country: 100.0% also missing
  Subregion: 100.0% also missing
  Primary_outcomes: 100.0% also mi

In [ ]:
# Cell 10: Investigate Studies with Findings but Missing Metadata
print("INVESTIGATING SUSPICIOUS RECORDS")
print("="*80)

# Find studies with findings but missing core metadata
has_findings_but_missing_data = consolidated[
    (consolidated['Findings'].notna()) &
    (consolidated['Abstract'].isna()) &
    (consolidated['Description_of_intervention'].isna())
]

print(f"Found {len(has_findings_but_missing_data)} studies with Findings but missing Abstract AND Intervention")

# Sample these problematic records
print("\nSample of suspicious records:")
print("-"*80)
for idx, row in has_findings_but_missing_data.head(5).iterrows():
    print(f"\nRCT ID: {row['rct_id']}")
    print(f"Title: {row['Title'][:60] if pd.notna(row['Title']) else 'NO TITLE'}")
    print(f"Researchers: {row['Researchers'][:60] if pd.notna(row['Researchers']) else 'NO RESEARCHERS'}")
    print(f"Findings: {row['Findings'][:150] if pd.notna(row['Findings']) else 'NO FINDINGS'}...")

# Check if these come from a specific source
if 'source_file' in df.columns:
    source_analysis = df[df['rct_id'].isin(has_findings_but_missing_data['rct_id'])]['source_file'].value_counts()
    print("\n" + "="*80)
    print("SOURCE FILE ANALYSIS:")
    print(source_analysis)

# Check the findings content - are they actually valid findings?
print("\n" + "="*80)
print("FINDINGS QUALITY CHECK:")
findings_lengths = has_findings_but_missing_data['Findings'].str.len()
print(f"Average findings length: {findings_lengths.mean():.0f} characters")
print(f"Median findings length: {findings_lengths.median():.0f} characters")
print(f"Min findings length: {findings_lengths.min():.0f} characters")

# Check if these might be from the Sonar harvesting that didn't get proper metadata
print("\n" + "="*80)
print("HYPOTHESIS: These might be from external harvesting without registry metadata")

# Check the original df for these problematic IDs
sample_ids = has_findings_but_missing_data['rct_id'].head(5).tolist()
for rct_id in sample_ids:
    if rct_id in df['rct_id'].values:
        original_row = df[df['rct_id'] == rct_id].iloc[0]
        print(f"\nRCT {rct_id} in original data:")
        print(f"  has_findings: {original_row.get('has_findings', 'N/A')}")
        print(f"  final_has_findings: {original_row.get('final_has_findings', 'N/A')}")
        print(f"  final_findings_source_type: {original_row.get('final_findings_source_type', 'N/A')}")

INVESTIGATING SUSPICIOUS RECORDS
Found 146 studies with Findings but missing Abstract AND Intervention

Sample of suspicious records:
--------------------------------------------------------------------------------

RCT ID: 1005.0
Title: Information Frictions in the Labor Market: Evidence from a F
Researchers: Vittorio Bassi
Findings: The study "Information Frictions in the Labor Market: Evidence from a Field Experiment in Uganda" by Vittorio Bassi finds that providing certification...

RCT ID: 1021.0
Title: Inefficient Hiring in Entry-Level Labor Markets
Researchers: Amanda Pallais
Findings: The study "Inefficient Hiring in Entry-Level Labor Markets" by Amanda Pallais reveals that hiring inexperienced workers and providing them with detail...

RCT ID: 1025.0
Title: Sports Betting in Uganda: Causes and Consequences
Researchers: Sylvan Herskowitz
Findings: The study by Sylvan Herskowitz found that sports betting in Uganda is primarily driven by unmet liquidity needs for large, indivisib

In [ ]:
# Cell 11: Trace Back and Recover Missing Metadata
print("TRACING MISSING METADATA FOR SONAR-HARVESTED STUDIES")
print("="*80)

# These 146 studies should exist in the original registry files
# Let's check if we can find them in the WITHOUT_findings file which was the input to Sonar

# Try loading the original WITHOUT_findings file that was fed to Sonar
without_findings_path = '/content/drive/MyDrive/AEA_RCT_Parsed/Main (9)_studies_WITHOUT_findings.csv'
try:
    df_without = pd.read_csv(without_findings_path)
    print(f"Loaded WITHOUT_findings file: {len(df_without)} rows")

    # Check if our problematic RCT IDs exist there
    problematic_ids = has_findings_but_missing_data['rct_id'].dropna().unique()
    found_in_original = df_without[df_without['rct_id'].isin(problematic_ids)]

    print(f"\nFound {len(found_in_original)} of {len(problematic_ids)} problematic IDs in original WITHOUT_findings file")

    if len(found_in_original) > 0:
        # Check what metadata exists in the original
        sample = found_in_original.iloc[0]
        print("\nSample record from original file:")
        print(f"RCT ID: {sample['rct_id']}")
        print(f"Title: {sample.get('title', 'NO TITLE')}")
        print(f"Abstract: {'Present' if pd.notna(sample.get('abstract')) and len(str(sample.get('abstract', ''))) > 10 else 'Missing'}")
        print(f"Country: {sample.get('country', 'Missing')}")
        print(f"Interventions: {'Present' if pd.notna(sample.get('interventions')) else 'Missing'}")

        # Recovery strategy: merge the original metadata with the findings
        print("\n" + "="*80)
        print("RECOVERY STRATEGY:")
        print("1. Merge original registry metadata from WITHOUT_findings file")
        print("2. Keep the harvested findings from sonar_append_all")
        print("3. This will restore the complete records")

except FileNotFoundError:
    print(f"Could not find: {without_findings_path}")
    print("\nAlternative: Try loading the SONAR results file directly")

    sonar_results_path = '/content/drive/MyDrive/AEA_RCT_Parsed/Main (9)_studies_WITHOUT_findings_PPLX_RESULTS_sonar.csv'
    try:
        df_sonar = pd.read_csv(sonar_results_path)
        print(f"Loaded SONAR results file: {len(df_sonar)} rows")
        print("Columns in SONAR file:", df_sonar.columns.tolist()[:10])  # Show first 10 columns
    except:
        print("Could not load SONAR results file either")

print("\n" + "="*80)
print("RECOMMENDATION:")
print("The issue is that sonar_append_all contains ONLY the harvested findings,")
print("not the original registry metadata. We need to merge these 146 records")
print("with their original registry data from the WITHOUT_findings input file.")

TRACING MISSING METADATA FOR SONAR-HARVESTED STUDIES
Loaded WITHOUT_findings file: 1936 rows

Found 0 of 145 problematic IDs in original WITHOUT_findings file

RECOMMENDATION:
The issue is that sonar_append_all contains ONLY the harvested findings,
not the original registry metadata. We need to merge these 146 records
with their original registry data from the WITHOUT_findings input file.


In [ ]:
# Cell 12: Find the True Source of These Mystery Studies
print("INVESTIGATING THE SOURCE OF MYSTERY STUDIES")
print("="*80)

# Check the RCT ID ranges to understand the pattern
problematic_ids = has_findings_but_missing_data['rct_id'].dropna()
print(f"RCT ID range of problematic studies: {problematic_ids.min():.0f} to {problematic_ids.max():.0f}")

# Check RCT ID ranges in the full dataset
all_rct_ids = df['rct_id'].dropna()
print(f"RCT ID range in full dataset: {all_rct_ids.min():.0f} to {all_rct_ids.max():.0f}")

# Look for these IDs in other possible source files
print("\n" + "="*80)
print("CHECKING OTHER POSSIBLE SOURCE FILES:")

possible_sources = [
    'Main (9)_studies_WITH_findings.csv',
    'studies_WITH_findings.csv',
    'all studies with findings.csv',
    'WITH_findings__combined_dedup.csv',
    'potential_misses_from_parsed_Main (9)_with_findings.csv'
]

for filename in possible_sources:
    filepath = f'/content/drive/MyDrive/AEA_RCT_Parsed/{filename}'
    try:
        temp_df = pd.read_csv(filepath)
        # Check if our problematic IDs exist in this file
        found_ids = temp_df[temp_df['rct_id'].isin(problematic_ids)]['rct_id'].nunique()
        if found_ids > 0:
            print(f"\n✓ {filename}: Found {found_ids} problematic IDs")
            # Check if these records have metadata
            sample = temp_df[temp_df['rct_id'].isin(problematic_ids)].iloc[0]
            has_abstract = pd.notna(sample.get('abstract')) and len(str(sample.get('abstract', ''))) > 10
            has_country = pd.notna(sample.get('country'))
            print(f"  Sample has abstract: {has_abstract}, has country: {has_country}")
        else:
            print(f"✗ {filename}: No problematic IDs found")
    except:
        print(f"✗ {filename}: File not found or couldn't load")

# Check if these are completely new studies that were never in the registry
print("\n" + "="*80)
print("HYPOTHESIS CHECK:")
print("These might be studies found ONLY through external search,")
print("never registered in AEA RCT Registry, but found during Sonar harvesting.")
print("\nIf so, we'd need to either:")
print("1. Remove them (they're not true AEA RCT registry studies)")
print("2. Try to recover their metadata from the external sources")
print("3. Keep them but mark as 'external only' studies")

INVESTIGATING THE SOURCE OF MYSTERY STUDIES
RCT ID range of problematic studies: 78 to 1772
RCT ID range in full dataset: 76 to 16284

CHECKING OTHER POSSIBLE SOURCE FILES:
✗ Main (9)_studies_WITH_findings.csv: No problematic IDs found
✗ studies_WITH_findings.csv: No problematic IDs found
✗ all studies with findings.csv: No problematic IDs found
✗ WITH_findings__combined_dedup.csv: No problematic IDs found
✗ potential_misses_from_parsed_Main (9)_with_findings.csv: No problematic IDs found

HYPOTHESIS CHECK:
These might be studies found ONLY through external search,
never registered in AEA RCT Registry, but found during Sonar harvesting.

If so, we'd need to either:
1. Remove them (they're not true AEA RCT registry studies)
2. Try to recover their metadata from the external sources
3. Keep them but mark as 'external only' studies


In [ ]:
# Cell 13: Check the OTHER WITHOUT_findings File
print("CHECKING ALTERNATIVE WITHOUT_FINDINGS FILE")
print("="*80)

# Try the other WITHOUT_findings file
alt_without_path = '/content/drive/MyDrive/AEA_RCT_Parsed/studies_WITHOUT_findings.csv'
try:
    df_alt_without = pd.read_csv(alt_without_path)
    print(f"Loaded studies_WITHOUT_findings.csv: {len(df_alt_without)} rows")
    print(f"RCT ID range: {df_alt_without['rct_id'].min():.0f} to {df_alt_without['rct_id'].max():.0f}")

    # Check if our problematic IDs are in this file
    problematic_ids = has_findings_but_missing_data['rct_id'].dropna().unique()
    found_in_alt = df_alt_without[df_alt_without['rct_id'].isin(problematic_ids)]

    print(f"\nFound {len(found_in_alt)} of {len(problematic_ids)} problematic IDs in studies_WITHOUT_findings.csv!")

    if len(found_in_alt) > 0:
        # Check sample metadata
        print("\nSample records with metadata:")
        for idx, row in found_in_alt.head(3).iterrows():
            print(f"\nRCT {row['rct_id']}:")
            print(f"  Title: {row.get('title', 'N/A')[:60]}")
            print(f"  Abstract: {'Present' if pd.notna(row.get('abstract')) and len(str(row.get('abstract'))) > 10 else 'Missing'}")
            print(f"  Country: {row.get('country', 'N/A')}")
            print(f"  Interventions: {'Present' if pd.notna(row.get('interventions')) and len(str(row.get('interventions'))) > 10 else 'Missing'}")

        print("\n" + "="*80)
        print("SOLUTION IDENTIFIED:")
        print("These 146 studies need their metadata recovered from studies_WITHOUT_findings.csv")
        print("The 'sonar_append_all' process only added findings but lost the original metadata.")

except Exception as e:
    print(f"Error loading file: {e}")
    print("\nCheck if 'studies_WITHOUT_findings.csv' exists in your folder")

CHECKING ALTERNATIVE WITHOUT_FINDINGS FILE
Loaded studies_WITHOUT_findings.csv: 268 rows
RCT ID range: 78 to 1773

Found 145 of 145 problematic IDs in studies_WITHOUT_findings.csv!

Sample records with metadata:

RCT 1772:
  Title: Improving school preparedness and child health outcomes thro
  Abstract: Present
  Country: India
  Interventions: Present

RCT 1748:
  Title: Nudging Farmers to Use Fertilizer: Experimental Evidence fro
  Abstract: Present
  Country: Kenya
  Interventions: Present

RCT 1647:
  Title: Farm & Family Balance Study: the intra-household and gender 
  Abstract: Present
  Country: Uganda
  Interventions: Present

SOLUTION IDENTIFIED:
These 146 studies need their metadata recovered from studies_WITHOUT_findings.csv
The 'sonar_append_all' process only added findings but lost the original metadata.


In [ ]:
# Cell 14: Recover Missing Metadata for Sonar Studies
print("RECOVERING METADATA FOR 146 STUDIES")
print("="*80)

# Get the full metadata from studies_WITHOUT_findings.csv
df_alt_without = pd.read_csv('/content/drive/MyDrive/AEA_RCT_Parsed/studies_WITHOUT_findings.csv')

# Identify rows in consolidated that need metadata recovery
needs_recovery = consolidated[
    (consolidated['Findings'].notna()) &
    (consolidated['Abstract'].isna()) &
    (consolidated['rct_id'].isin(df_alt_without['rct_id']))
].copy()

print(f"Recovering metadata for {len(needs_recovery)} studies")

# Create mapping dictionary from studies_WITHOUT_findings
metadata_map = {}
for _, row in df_alt_without.iterrows():
    rct_id = row['rct_id']
    metadata_map[rct_id] = {
        'Abstract': row.get('abstract'),
        'Country': row.get('country'),
        'Subregion': row.get('region'),
        'Description_of_intervention': row.get('interventions'),
        'Primary_outcomes': row.get('primary_outcomes_endpoints'),
        'Secondary_outcomes': row.get('secondary_outcomes_endpoints'),
        'Year': extract_year(row),  # Using the function we defined earlier
        'Researcher_affiliation': row.get('primary_investigator_affiliation'),
        'Population': row.get('population_extracted'),
        'jel_codes': row.get('jel_codes'),
        'keywords': row.get('keywords'),
        'keywords_additional': row.get('keywords_additional')
    }

# Apply recovery to consolidated dataset
recovery_count = 0
for idx in consolidated.index:
    rct_id = consolidated.at[idx, 'rct_id']
    if rct_id in metadata_map and pd.isna(consolidated.at[idx, 'Abstract']):
        # This row needs recovery
        for field, value in metadata_map[rct_id].items():
            if field in consolidated.columns:
                consolidated.at[idx, field] = value
        recovery_count += 1

print(f"Successfully recovered metadata for {recovery_count} studies")

# Verify recovery
print("\n" + "="*80)
print("POST-RECOVERY STATISTICS:")

# Recalculate missing data
key_fields = ['Title', 'Abstract', 'Country', 'Description_of_intervention', 'Year']
for field in key_fields:
    before_missing = has_findings_but_missing_data[field].isna().sum()
    after_missing = consolidated[field].isna().sum()
    print(f"{field}: {after_missing} missing (was {before_missing + (len(consolidated) - len(has_findings_but_missing_data))})")

# Check if we still have studies with findings but no metadata
still_problematic = consolidated[
    (consolidated['Findings'].notna()) &
    (consolidated['Abstract'].isna()) &
    (consolidated['Description_of_intervention'].isna())
]

print(f"\nStudies with findings but still missing core metadata: {len(still_problematic)}")

if len(still_problematic) > 0:
    print("Sample of remaining problematic studies:")
    for idx, row in still_problematic.head(3).iterrows():
        print(f"  RCT {row['rct_id']}: {row['Title'][:50] if pd.notna(row['Title']) else 'NO TITLE'}")

RECOVERING METADATA FOR 146 STUDIES
Recovering metadata for 145 studies
Successfully recovered metadata for 145 studies

POST-RECOVERY STATISTICS:
Title: 2 missing (was 2137)
Abstract: 7 missing (was 2282)
Country: 3 missing (was 2282)
Description_of_intervention: 1 missing (was 2282)
Year: 1 missing (was 2282)

Studies with findings but still missing core metadata: 1
Sample of remaining problematic studies:
  RCT nan: NO TITLE


In [ ]:
# Cell 15: Clean Final Dataset and Save
print("FINAL DATASET CLEANUP AND EXPORT")
print("="*80)

# Remove the row with nan rct_id (likely an empty row)
consolidated_clean = consolidated[consolidated['rct_id'].notna()].copy()

# Remove the temporary 'missing_count' column if it exists
if 'missing_count' in consolidated_clean.columns:
    consolidated_clean = consolidated_clean.drop(columns=['missing_count'])

print(f"Removed {len(consolidated) - len(consolidated_clean)} empty/invalid rows")
print(f"Final dataset: {len(consolidated_clean)} studies")

# Final quality report
print("\n" + "="*80)
print("FINAL DATASET QUALITY REPORT:")
print(f"Total studies: {len(consolidated_clean)}")
print(f"Unique RCT IDs: {consolidated_clean['rct_id'].nunique()}")
print(f"Studies with complete core fields: {len(consolidated_clean.dropna(subset=['Title', 'Abstract', 'Description_of_intervention', 'Findings']))}")

# Population rates for key fields
print("\nField completion rates:")
important_fields = ['Title', 'Researchers', 'Abstract', 'Description_of_intervention',
                   'Primary_outcomes', 'Findings', 'Country', 'Year']
for field in important_fields:
    populated = consolidated_clean[field].notna().sum()
    pct = (populated / len(consolidated_clean)) * 100
    print(f"  {field}: {pct:.1f}%")

# Save the consolidated dataset
output_path = '/content/drive/MyDrive/AEA_RCT_Parsed/consolidated_rct_dataset_clean.csv'
consolidated_clean.to_csv(output_path, index=False)
print(f"\n✓ Saved consolidated dataset to: consolidated_rct_dataset_clean.csv")

# Create a subset without rct_id for Azure (if needed)
consolidated_for_azure = consolidated_clean.drop(columns=['rct_id'])
azure_output_path = '/content/drive/MyDrive/AEA_RCT_Parsed/consolidated_rct_dataset_azure.csv'
consolidated_for_azure.to_csv(azure_output_path, index=False)
print(f"✓ Saved Azure-ready dataset (without rct_id) to: consolidated_rct_dataset_azure.csv")

print(f"\nDatasets ready with {len(consolidated_clean.columns)-1} searchable fields for Azure AI Search")

FINAL DATASET CLEANUP AND EXPORT
Removed 1 empty/invalid rows
Final dataset: 2281 studies

FINAL DATASET QUALITY REPORT:
Total studies: 2281
Unique RCT IDs: 2281
Studies with complete core fields: 2274

Field completion rates:
  Title: 100.0%
  Researchers: 100.0%
  Abstract: 99.7%
  Description_of_intervention: 100.0%
  Primary_outcomes: 100.0%
  Findings: 100.0%
  Country: 99.9%
  Year: 100.0%

✓ Saved consolidated dataset to: consolidated_rct_dataset_clean.csv
✓ Saved Azure-ready dataset (without rct_id) to: consolidated_rct_dataset_azure.csv

Datasets ready with 20 searchable fields for Azure AI Search


In [ ]:
# Cell 16: Comprehensive Field Analysis
print("COMPREHENSIVE FIELD ANALYSIS")
print("="*80)

# Analyze all fields in the dataset
all_fields_stats = []
for col in consolidated_clean.columns:
    populated = consolidated_clean[col].notna().sum()
    empty = consolidated_clean[col].isna().sum()
    pct_populated = (populated / len(consolidated_clean)) * 100

    # Get sample length for text fields
    if consolidated_clean[col].dtype == 'object':
        non_null = consolidated_clean[col].dropna()
        if len(non_null) > 0:
            avg_length = non_null.str.len().mean()
            median_length = non_null.str.len().median()
        else:
            avg_length = 0
            median_length = 0
    else:
        avg_length = None
        median_length = None

    all_fields_stats.append({
        'Field': col,
        'Populated': populated,
        'Empty': empty,
        'Completion_Rate': f"{pct_populated:.1f}%",
        'Avg_Length': f"{avg_length:.0f}" if avg_length else "N/A",
        'Median_Length': f"{median_length:.0f}" if median_length else "N/A"
    })

# Display as DataFrame for better readability
stats_df = pd.DataFrame(all_fields_stats)
print(stats_df.to_string(index=False))

print("\n" + "="*80)
print("FIELD CATEGORIES SUMMARY:")

# Group fields by completion rate
print("\nFully populated (100%):")
fully_populated = stats_df[stats_df['Completion_Rate'] == '100.0%']['Field'].tolist()
print(f"  {', '.join(fully_populated)}")

print("\nNearly complete (>95%):")
nearly_complete = stats_df[(stats_df['Completion_Rate'] != '100.0%') &
                           (stats_df['Populated'] / len(consolidated_clean) > 0.95)]['Field'].tolist()
print(f"  {', '.join(nearly_complete)}")

print("\nModerate completion (50-95%):")
moderate = stats_df[(stats_df['Populated'] / len(consolidated_clean) <= 0.95) &
                    (stats_df['Populated'] / len(consolidated_clean) > 0.50)]['Field'].tolist()
print(f"  {', '.join(moderate)}")

print("\nLow completion (<50%):")
low_completion = stats_df[stats_df['Populated'] / len(consolidated_clean) <= 0.50]['Field'].tolist()
print(f"  {', '.join(low_completion)}")

# Check for fields that might need attention
print("\n" + "="*80)
print("FIELDS REQUIRING ATTENTION:")
problematic = stats_df[stats_df['Populated'] / len(consolidated_clean) < 0.90]
if len(problematic) > 0:
    print(f"\nFields with <90% completion that might benefit from enrichment:")
    for _, row in problematic.iterrows():
        print(f"  - {row['Field']}: {row['Completion_Rate']} complete ({row['Empty']} missing)")
else:
    print("All fields have >90% completion rate")

COMPREHENSIVE FIELD ANALYSIS
                      Field  Populated  Empty Completion_Rate Avg_Length Median_Length
                      Title       2280      1          100.0%         73            71
                    Country       2279      2           99.9%         14             8
                  Subregion       1089   1192           47.7%         16            10
                   Abstract       2275      6           99.7%       1028           876
Description_of_intervention       2281      0          100.0%       1053           637
           Primary_outcomes       2281      0          100.0%        754           395
         Secondary_outcomes       2281      0          100.0%        176            32
                 Population       2110    171           92.5%         65            63
              Search_vector       2281      0          100.0%      21104         22704
                   keywords       2281      0          100.0%         18            15
        keywor

In [ ]:
# Cell 17: Analyze Studies Missing AI-Generated Keywords
print("ANALYSIS OF STUDIES MISSING AI-GENERATED KEYWORDS")
print("="*80)

# Identify studies missing the keyword subcategories
keyword_subcats = ['keywords_sector', 'keywords_mechanisms', 'keywords_implementation', 'keywords_context']

# Find studies missing ANY of these subcategories
missing_keywords_mask = consolidated_clean[keyword_subcats].isna().any(axis=1)
studies_missing_keywords = consolidated_clean[missing_keywords_mask].copy()

print(f"Studies missing at least one keyword subcategory: {len(studies_missing_keywords)}")

# Check if they're missing ALL subcategories or just some
missing_all_subcats = consolidated_clean[keyword_subcats].isna().all(axis=1)
print(f"Studies missing ALL keyword subcategories: {missing_all_subcats.sum()}")

# Analyze patterns in missing keywords
print("\n" + "="*80)
print("PATTERNS IN STUDIES MISSING KEYWORDS:")

# Check if these are the same studies missing other fields
print("\nCross-checking with other missing fields:")
for field in ['Abstract', 'Population', 'jel_codes']:
    missing_both = (missing_keywords_mask & consolidated_clean[field].isna()).sum()
    print(f"  Missing keywords AND {field}: {missing_both}")

# Check RCT ID ranges
print(f"\nRCT ID range of studies missing keywords:")
missing_ids = studies_missing_keywords['rct_id'].dropna()
print(f"  Min: {missing_ids.min():.0f}, Max: {missing_ids.max():.0f}")

# Check if these are from a specific data source
print("\nSample of studies missing keywords:")
sample_missing = studies_missing_keywords.head(10)
for idx, row in sample_missing.iterrows():
    has_abstract = 'Yes' if pd.notna(row['Abstract']) and len(str(row['Abstract'])) > 10 else 'No'
    has_population = 'Yes' if pd.notna(row['Population']) else 'No'
    print(f"\nRCT {row['rct_id']}: {row['Title'][:50]}")
    print(f"  Has abstract: {has_abstract}, Has population: {has_population}")
    print(f"  Country: {row['Country']}, Year: {row['Year']}")

# Check if we can generate keywords for these using LLM
print("\n" + "="*80)
print("FEASIBILITY CHECK FOR KEYWORD GENERATION:")

# Studies with abstract but missing keywords
has_abstract_no_keywords = studies_missing_keywords[
    studies_missing_keywords['Abstract'].notna() &
    (studies_missing_keywords['Abstract'].str.len() > 100)
]

print(f"Studies with good abstracts but missing keywords: {len(has_abstract_no_keywords)}")
print("These could have keywords generated using LLM based on their abstracts")

# Studies without sufficient text for keyword generation
no_text_for_keywords = studies_missing_keywords[
    studies_missing_keywords['Abstract'].isna() |
    (studies_missing_keywords['Abstract'].str.len() <= 100)
]

print(f"Studies without sufficient text for keyword generation: {len(no_text_for_keywords)}")
print("These would need alternative approaches or remain without subcategory keywords")

ANALYSIS OF STUDIES MISSING AI-GENERATED KEYWORDS
Studies missing at least one keyword subcategory: 167
Studies missing ALL keyword subcategories: 164

PATTERNS IN STUDIES MISSING KEYWORDS:

Cross-checking with other missing fields:
  Missing keywords AND Abstract: 2
  Missing keywords AND Population: 167
  Missing keywords AND jel_codes: 92

RCT ID range of studies missing keywords:
  Min: 78, Max: 14794

Sample of studies missing keywords:

RCT 2417.0: Beliefs about Effectiveness and Political Activism
  Has abstract: Yes, Has population: No
  Country: Germany, Year: 2017

RCT 2644.0: Marriage Registrars Training
  Has abstract: Yes, Has population: No
  Country: Pakistan, Year: 2017

RCT 2690.0: Incentives and education
  Has abstract: Yes, Has population: No
  Country: Spain, Year: 2018

RCT 3333.0: An audit study on minimum wage legislation
  Has abstract: Yes, Has population: No
  Country: United States of America, Year: 2018

RCT 3507.0: The effects of self-chosen incentivized g

In [ ]:
# Cell 18: Analyze Abstract Length for Studies Missing Keywords
print("ABSTRACT LENGTH ANALYSIS FOR STUDIES MISSING KEYWORDS")
print("="*80)

# Get abstract lengths for studies WITH keywords
has_keywords_mask = ~missing_keywords_mask
with_keywords = consolidated_clean[has_keywords_mask].copy()
with_keywords['abstract_length'] = with_keywords['Abstract'].fillna('').str.len()

# Get abstract lengths for studies WITHOUT keywords
without_keywords = studies_missing_keywords.copy()
without_keywords['abstract_length'] = without_keywords['Abstract'].fillna('').str.len()

# Compare statistics
print("Abstract length statistics:")
print("-"*40)
print(f"Studies WITH keywords (n={len(with_keywords)}):")
print(f"  Mean length: {with_keywords['abstract_length'].mean():.0f} characters")
print(f"  Median length: {with_keywords['abstract_length'].median():.0f} characters")
print(f"  Min length: {with_keywords['abstract_length'].min():.0f} characters")
print(f"  Max length: {with_keywords['abstract_length'].max():.0f} characters")
print(f"  Abstracts <100 chars: {(with_keywords['abstract_length'] < 100).sum()}")
print(f"  Abstracts <500 chars: {(with_keywords['abstract_length'] < 500).sum()}")

print(f"\nStudies WITHOUT keywords (n={len(without_keywords)}):")
print(f"  Mean length: {without_keywords['abstract_length'].mean():.0f} characters")
print(f"  Median length: {without_keywords['abstract_length'].median():.0f} characters")
print(f"  Min length: {without_keywords['abstract_length'].min():.0f} characters")
print(f"  Max length: {without_keywords['abstract_length'].max():.0f} characters")
print(f"  Abstracts <100 chars: {(without_keywords['abstract_length'] < 100).sum()}")
print(f"  Abstracts <500 chars: {(without_keywords['abstract_length'] < 500).sum()}")

# Distribution of abstract lengths for missing keywords
print("\n" + "="*80)
print("ABSTRACT LENGTH DISTRIBUTION FOR STUDIES MISSING KEYWORDS:")
length_bins = [0, 100, 250, 500, 750, 1000, 10000]
length_labels = ['0-100', '100-250', '250-500', '500-750', '750-1000', '1000+']
without_keywords['length_category'] = pd.cut(without_keywords['abstract_length'],
                                              bins=length_bins,
                                              labels=length_labels)

for category in length_labels:
    count = (without_keywords['length_category'] == category).sum()
    pct = (count / len(without_keywords)) * 100
    print(f"  {category} chars: {count} studies ({pct:.1f}%)")

# Show examples of different length categories
print("\n" + "="*80)
print("SAMPLE ABSTRACTS FROM STUDIES MISSING KEYWORDS:")

for category in ['0-100', '100-250', '250-500']:
    sample = without_keywords[without_keywords['length_category'] == category].head(1)
    if len(sample) > 0:
        row = sample.iloc[0]
        print(f"\n{category} chars example (RCT {row['rct_id']}):")
        print(f"  Title: {row['Title'][:60]}")
        print(f"  Abstract: '{row['Abstract'][:200] if pd.notna(row['Abstract']) else 'EMPTY'}'")
        print(f"  Length: {row['abstract_length']} characters")

# Check if there's a clear cutoff threshold
print("\n" + "="*80)
print("THRESHOLD ANALYSIS:")
print("Is there a minimum abstract length used for keyword generation?")

# Find the shortest abstract that HAS keywords
min_length_with_keywords = with_keywords[with_keywords['abstract_length'] > 0]['abstract_length'].min()
print(f"  Shortest abstract WITH keywords: {min_length_with_keywords:.0f} chars")

# Find the longest abstract WITHOUT keywords
max_length_without_keywords = without_keywords[without_keywords['abstract_length'] > 0]['abstract_length'].max()
print(f"  Longest abstract WITHOUT keywords: {max_length_without_keywords:.0f} chars")

if min_length_with_keywords < max_length_without_keywords:
    print(f"\n  ⚠️ There's overlap: some abstracts with {min_length_with_keywords:.0f}+ chars have keywords")
    print(f"     while some with up to {max_length_without_keywords:.0f} chars don't.")
    print("     The missing keywords might not be purely due to abstract length.")

ABSTRACT LENGTH ANALYSIS FOR STUDIES MISSING KEYWORDS
Abstract length statistics:
----------------------------------------
Studies WITH keywords (n=2114):
  Mean length: 1032 characters
  Median length: 877 characters
  Min length: 0 characters
  Max length: 9713 characters
  Abstracts <100 chars: 16
  Abstracts <500 chars: 338

Studies WITHOUT keywords (n=167):
  Mean length: 945 characters
  Median length: 812 characters
  Min length: 0 characters
  Max length: 3586 characters
  Abstracts <100 chars: 16
  Abstracts <500 chars: 43

ABSTRACT LENGTH DISTRIBUTION FOR STUDIES MISSING KEYWORDS:
  0-100 chars: 15 studies (9.0%)
  100-250 chars: 13 studies (7.8%)
  250-500 chars: 13 studies (7.8%)
  500-750 chars: 28 studies (16.8%)
  750-1000 chars: 29 studies (17.4%)
  1000+ chars: 67 studies (40.1%)

SAMPLE ABSTRACTS FROM STUDIES MISSING KEYWORDS:

0-100 chars example (RCT 2417.0):
  Title: Beliefs about Effectiveness and Political Activism: Evidence
  Abstract: 'See preanalysis plan.
Ext

In [ ]:
# Cell 19: Investigate the Real Pattern Behind Missing Keywords
print("INVESTIGATING TRUE PATTERN FOR MISSING KEYWORDS")
print("="*80)

# Check the actual content quality of abstracts without keywords
without_keywords = studies_missing_keywords.copy()

# Identify potential placeholder/low-quality abstracts
placeholder_patterns = [
    'see preanalysis',
    'see pre-analysis',
    'external link',
    'uploaded all information',
    'pdf file',
    'will remain hidden',
    'see attached',
    'see appendix',
    'forthcoming',
    'to be provided'
]

# Check for placeholder text
def has_placeholder_text(text):
    if pd.isna(text):
        return True
    text_lower = str(text).lower()
    return any(pattern in text_lower for pattern in placeholder_patterns)

without_keywords['has_placeholder'] = without_keywords['Abstract'].apply(has_placeholder_text)

print(f"Studies with placeholder/incomplete abstracts: {without_keywords['has_placeholder'].sum()}")
print(f"Studies with real abstracts: {(~without_keywords['has_placeholder']).sum()}")

# Also check if these are from a specific time period or batch
print("\n" + "="*80)
print("TEMPORAL PATTERN ANALYSIS:")

# Year distribution
year_dist = without_keywords.groupby('Year').size().sort_index()
print("\nStudies missing keywords by year:")
for year, count in year_dist.items():
    if count > 5:  # Only show years with significant counts
        print(f"  {year}: {count} studies")

# Check if these are from the studies_WITHOUT_findings batch
print("\n" + "="*80)
print("SOURCE ANALYSIS:")

# Check RCT ID ranges to identify source
id_ranges = [
    (78, 1773, "studies_WITHOUT_findings"),
    (2000, 17000, "Main batch")
]

for min_id, max_id, source in id_ranges:
    in_range = without_keywords[(without_keywords['rct_id'] >= min_id) &
                                (without_keywords['rct_id'] <= max_id)]
    if len(in_range) > 0:
        print(f"{source} (ID {min_id}-{max_id}): {len(in_range)} studies missing keywords")

# Sample real abstracts that are missing keywords
print("\n" + "="*80)
print("SAMPLE OF REAL ABSTRACTS MISSING KEYWORDS:")

real_abstracts = without_keywords[(~without_keywords['has_placeholder']) &
                                  (without_keywords['Abstract'].str.len() > 500)]

for idx, row in real_abstracts.head(3).iterrows():
    print(f"\nRCT {row['rct_id']}: {row['Title'][:60]}")
    print(f"Abstract preview: {row['Abstract'][:300]}...")
    print(f"Year: {row['Year']}, Country: {row['Country']}")

# Check if Population field correlates (all missing keywords lack Population)
print("\n" + "="*80)
print("KEY INSIGHT:")
print(f"ALL {len(without_keywords)} studies missing keywords also lack 'Population' field")
print("This suggests they weren't processed by the AI enrichment pipeline that generated both")
print("\nRecommendation: Run AI enrichment on these {len(real_abstracts)} studies with real abstracts")

INVESTIGATING TRUE PATTERN FOR MISSING KEYWORDS
Studies with placeholder/incomplete abstracts: 154
Studies with real abstracts: 13

TEMPORAL PATTERN ANALYSIS:

Studies missing keywords by year:
  2009: 7 studies
  2010: 10 studies
  2011: 12 studies
  2013: 14 studies
  2014: 30 studies
  2015: 25 studies
  2016: 26 studies
  2019: 6 studies

SOURCE ANALYSIS:
studies_WITHOUT_findings (ID 78-1773): 145 studies missing keywords
Main batch (ID 2000-17000): 22 studies missing keywords

SAMPLE OF REAL ABSTRACTS MISSING KEYWORDS:

RCT 1243.0: The Impacts of Female Education: Evidence from Malawian Seco
Abstract preview: This project evaluates a randomized controlled trial of an education support program for girls on rationality and preferences. Between October of 2011 and May of 2012 a baseline survey of 7,971 secondary students in school cohorts 9-11 was implemented across 124 classrooms in 33 public secondary sch...
Year: 2011, Country: Malawi

RCT 1516.0: A randomized controlled trial of 

In [ ]:
# Cell 20: Final Export and Documentation
print("FINAL EXPORT AND ENRICHMENT RECOMMENDATIONS")
print("="*80)

# Create a list of studies that need AI enrichment
needs_enrichment = without_keywords[(~without_keywords['has_placeholder']) &
                                    (without_keywords['Abstract'].str.len() > 300)]

print(f"Studies suitable for AI enrichment: {len(needs_enrichment)}")

# Save list of studies needing enrichment
enrichment_candidates = needs_enrichment[['rct_id', 'Title', 'Abstract', 'Year', 'Country']].copy()
enrichment_candidates['abstract_length'] = enrichment_candidates['Abstract'].str.len()
enrichment_candidates['needs'] = 'Population, keywords_sector, keywords_mechanisms, keywords_implementation, keywords_context'

enrichment_path = '/content/drive/MyDrive/AEA_RCT_Parsed/studies_for_ai_enrichment.csv'
enrichment_candidates.to_csv(enrichment_path, index=False)
print(f"✓ Saved enrichment candidates to: studies_for_ai_enrichment.csv")

# Final summary statistics
print("\n" + "="*80)
print("FINAL CONSOLIDATED DATASET SUMMARY:")
print(f"""
Total Studies: {len(consolidated_clean)}
Complete Records (all major fields): {len(consolidated_clean.dropna(subset=['Title', 'Abstract', 'Description_of_intervention', 'Findings']))}

Field Completion Rates:
- Core Content: 99.7% (Abstract, Interventions, Outcomes, Findings)
- Geographic: 99.9% (Country), 47.7% (Subregion)
- Temporal: 100% (Year extracted from dates)
- AI-Enhanced: 92.7% (Keyword subcategories), 92.5% (Population)
- Traditional: 56.9% (JEL codes)

Data Quality Issues Resolved:
✓ Removed {len(df) - len(consolidated_clean)} duplicate studies
✓ Recovered metadata for 145 studies from Sonar harvesting
✓ Consolidated researcher names and affiliations
✓ Standardized all date fields to Year

Ready for Azure AI Search:
- 20 searchable fields
- 100% search vector coverage
- Rich keyword taxonomy for faceting
""")

print("="*80)
print("NEXT STEPS:")
print("""
1. Upload 'consolidated_rct_dataset_azure.csv' to Azure AI Search
2. Configure index with:
   - Searchable: Title, Abstract, Description_of_intervention, Findings
   - Filterable: Country, Year, keywords_sector, keywords_context
   - Facetable: Country, Year, Subregion (where available)
   - Vectorizable: Search_vector field (already computed)

3. Optional: Run AI enrichment on 13 studies in 'studies_for_ai_enrichment.csv'
   to add missing Population and keyword subcategories
""")

FINAL EXPORT AND ENRICHMENT RECOMMENDATIONS
Studies suitable for AI enrichment: 8
✓ Saved enrichment candidates to: studies_for_ai_enrichment.csv

FINAL CONSOLIDATED DATASET SUMMARY:

Total Studies: 2281
Complete Records (all major fields): 2274

Field Completion Rates:
- Core Content: 99.7% (Abstract, Interventions, Outcomes, Findings)  
- Geographic: 99.9% (Country), 47.7% (Subregion)
- Temporal: 100% (Year extracted from dates)
- AI-Enhanced: 92.7% (Keyword subcategories), 92.5% (Population)
- Traditional: 56.9% (JEL codes)

Data Quality Issues Resolved:
✓ Removed 1 duplicate studies
✓ Recovered metadata for 145 studies from Sonar harvesting
✓ Consolidated researcher names and affiliations
✓ Standardized all date fields to Year

Ready for Azure AI Search:
- 20 searchable fields
- 100% search vector coverage
- Rich keyword taxonomy for faceting

NEXT STEPS:

1. Upload 'consolidated_rct_dataset_azure.csv' to Azure AI Search
2. Configure index with:
   - Searchable: Title, Abstract, De

In [ ]:
import os
from google.colab import userdata

# Retrieve the Together AI API key from Colab secrets
together_api_key = userdata.get('TOGETHER_API_KEY')

# Set the API key as an environment variable
os.environ['TOGETHER_API_KEY'] = together_api_key

print("Together AI API key loaded and set as environment variable.")

Together AI API key loaded and set as environment variable.


In [ ]:

import pandas as pd
import numpy as np
import json
import time
from typing import Dict, List, Tuple
import requests
from tqdm import tqdm
import os
from datetime import datetime

# Install required packages
!pip install together --quiet
!pip install tiktoken --quiet  # For token counting

import together
import tiktoken

# Set up TogetherAI
print("Setting up TogetherAI...")
# You'll need to set your API key - choose one method:

# Method 1: Direct input (for testing)
# together.api_key = "YOUR_API_KEY_HERE"

# Method 2: From Google Colab secrets (recommended)
# from google.colab import userdata
# together.api_key = userdata.get('TOGETHER_API_KEY')

# Method 3: Environment variable
# os.environ['TOGETHER_API_KEY'] = "YOUR_API_KEY_HERE"
# together.api_key = os.environ.get('TOGETHER_API_KEY')

# Method 4: Input prompt (secure)
import getpass
together.api_key = getpass.getpass('Enter your TogetherAI API key: ')

print("✓ TogetherAI configured")

# Base path for data
base_path = '/content/drive/MyDrive/AEA_RCT_Parsed/'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.6 MB/s eta 0:00:00
Setting up TogetherAI...
Enter your TogetherAI API key: ··········
✓ TogetherAI configured


In [ ]:
print("Loading consolidated dataset...")
df = pd.read_csv(f'{base_path}consolidated_rct_dataset_clean.csv')
print(f"✓ Loaded {len(df)} studies")

# Create subset for validation (you can adjust the sample size)
print("\nPreparing validation sample...")

# Strategy 1: Random sample
sample_size = 100  # Adjust based on your API budget
validation_df = df.sample(n=min(sample_size, len(df)), random_state=42).copy()

# Strategy 2: Focus on suspicious cases (uncomment to use)
# Look for studies with very short abstracts but long findings, or vice versa
# df['abstract_len'] = df['Abstract'].fillna('').str.len()
# df['findings_len'] = df['Findings'].fillna('').str.len()
# suspicious = df[
#     ((df['abstract_len'] < 100) & (df['findings_len'] > 500)) |
#     ((df['abstract_len'] > 1000) & (df['findings_len'] < 100))
# ]
# validation_df = suspicious.sample(n=min(sample_size, len(suspicious)), random_state=42).copy()

print(f"✓ Selected {len(validation_df)} studies for validation")

Loading consolidated dataset...
✓ Loaded 2281 studies

Preparing validation sample...
✓ Selected 100 studies for validation


In [ ]:
def create_validation_prompt(row: pd.Series) -> str:
    """Create a prompt to validate if findings match the study."""

    prompt = f"""You are a research validator. Analyze if the findings below correctly match the study description.

STUDY TITLE: {row.get('Title', 'N/A')}

ABSTRACT: {row.get('Abstract', 'N/A')[:500]}

DESCRIPTION OF INTERVENTION: {row.get('Description_of_intervention', 'N/A')[:300]}

PRIMARY OUTCOMES: {row.get('Primary_outcomes', 'N/A')[:300]}

RESEARCHERS: {row.get('Researchers', 'N/A')}

YEAR: {row.get('Year', 'N/A')}

COUNTRY: {row.get('Country', 'N/A')}

FINDINGS: {row.get('Findings', 'N/A')[:800]}

TASK: Evaluate if these findings logically match this study. Consider:
1. Do the findings discuss the same intervention described in the abstract?
2. Do the outcomes in findings match the primary outcomes listed?
3. Is the geographic context consistent?
4. Do the findings seem to be from a completely different study?

Respond with ONLY a JSON object in this format:
{{
    "match_score": <0-100 score, 100 means perfect match>,
    "confidence": <"high", "medium", or "low">,
    "issues": [<list any major discrepancies>],
    "verdict": <"correct_match", "likely_match", "uncertain", "likely_mismatch", or "definite_mismatch">
}}

Do not include any other text, only the JSON object."""

    return prompt

def create_cross_check_prompt(row: pd.Series) -> str:
    """Create a simpler cross-check prompt."""

    prompt = f"""Compare this study title and findings. Are they from the same research?

TITLE: {row.get('Title', 'N/A')}
FINDINGS EXCERPT: {row.get('Findings', 'N/A')[:400]}

Answer with only: YES (same study), NO (different study), or UNCERTAIN (cannot determine).
Then provide a confidence score 0-100.

Format: <YES/NO/UNCERTAIN> <SCORE>
Example: YES 95"""

    return prompt

In [ ]:
# Available open-source models on TogetherAI (choose based on your needs)
MODELS = {
    'mixtral-8x7b': 'mistralai/Mixtral-8x7B-Instruct-v0.1',  # Good balance
    'mixtral-8x22b': 'mistralai/Mixtral-8x22B-Instruct-v0.1',  # More accurate
    'llama-70b': 'meta-llama/Llama-3-70b-chat-hf',  # Very capable
    'llama-8b': 'meta-llama/Llama-3-8b-chat-hf',  # Faster, cheaper
    'qwen-72b': 'Qwen/Qwen2-72B-Instruct',  # Good for analysis
    'deepseek': 'deepseek-ai/deepseek-llm-67b-chat',  # Cost-effective
}

# Select model (change as needed)
selected_model = MODELS['mixtral-8x7b']  # Start with Mixtral for good balance
print(f"Using model: {selected_model}")

# Test the API connection
try:
    test_response = together.Complete.create(
        prompt="Respond with 'OK' if you're working.",
        model=selected_model,
        max_tokens=10
    )
    print(f"✓ API connection successful")
except Exception as e:
    print(f"❌ API connection failed: {e}")
    print("Please check your API key and connection")

Using model: mistralai/Mixtral-8x7B-Instruct-v0.1


/tmp/ipython-input-3800310509.py:17: DeprecationWarning: Call to deprecated function create.
  test_response = together.Complete.create(
/usr/local/lib/python3.12/dist-packages/together/legacy/complete.py:23: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment variable instead.
  warnings.warn(API_KEY_WARNING)


✓ API connection successful


In [ ]:
def validate_single_entry(row: pd.Series, model: str, max_retries: int = 3) -> Dict:
    """Validate a single study-findings pair."""

    prompt = create_validation_prompt(row)

    for attempt in range(max_retries):
        try:
            response = together.Complete.create(
                prompt=prompt,
                model=model,
                max_tokens=300,
                temperature=0.1,  # Low temperature for consistency
                stop=["```", "\n\n\n"]
            )

            result_text = response['output']['choices'][0]['text'].strip()

            # Try to parse JSON
            # Clean up common issues
            result_text = result_text.replace('```json', '').replace('```', '').strip()
            if result_text.startswith('{'):
                end_idx = result_text.rfind('}')
                if end_idx != -1:
                    result_text = result_text[:end_idx + 1]

            result = json.loads(result_text)
            result['raw_response'] = result_text
            result['model'] = model
            result['success'] = True
            return result

        except json.JSONDecodeError as e:
            if attempt < max_retries - 1:
                time.sleep(2)  # Rate limiting
                continue
            return {
                'match_score': -1,
                'confidence': 'error',
                'issues': [f'JSON parse error: {str(e)}'],
                'verdict': 'error',
                'raw_response': result_text if 'result_text' in locals() else 'No response',
                'success': False
            }
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2)
                continue
            return {
                'match_score': -1,
                'confidence': 'error',
                'issues': [f'API error: {str(e)}'],
                'verdict': 'error',
                'success': False
            }

    return {'success': False, 'verdict': 'error'}

def quick_cross_check(row: pd.Series, model: str) -> Tuple[str, int]:
    """Perform a quick cross-check with simpler prompt."""

    prompt = create_cross_check_prompt(row)

    try:
        response = together.Complete.create(
            prompt=prompt,
            model=model,
            max_tokens=20,
            temperature=0.1
        )

        result = response['output']['choices'][0]['text'].strip()
        parts = result.split()
        if len(parts) >= 2:
            verdict = parts[0]
            score = int(parts[1])
            return verdict, score
        return "UNCERTAIN", 50

    except Exception as e:
        return "ERROR", -1

In [ ]:
print("=" * 80)
print("STARTING VALIDATION PROCESS")
print("=" * 80)
print(f"Validating {len(validation_df)} studies...")
print(f"Estimated time: {len(validation_df) * 3} seconds")
print(f"Estimated cost: ~${len(validation_df) * 0.001:.2f} (varies by model)")

# Add columns for results
validation_df['validation_score'] = None
validation_df['validation_verdict'] = None
validation_df['validation_confidence'] = None
validation_df['validation_issues'] = None
validation_df['cross_check_verdict'] = None
validation_df['cross_check_score'] = None

# Progress tracking
validated_count = 0
error_count = 0
mismatch_count = 0

# Run validation with progress bar
for idx, row in tqdm(validation_df.iterrows(), total=len(validation_df), desc="Validating"):

    # Main validation
    result = validate_single_entry(row, selected_model)

    if result['success']:
        validation_df.at[idx, 'validation_score'] = result['match_score']
        validation_df.at[idx, 'validation_verdict'] = result['verdict']
        validation_df.at[idx, 'validation_confidence'] = result['confidence']
        validation_df.at[idx, 'validation_issues'] = json.dumps(result.get('issues', []))

        if result['verdict'] in ['likely_mismatch', 'definite_mismatch']:
            mismatch_count += 1

        validated_count += 1
    else:
        error_count += 1
        validation_df.at[idx, 'validation_verdict'] = 'error'

    # Quick cross-check for suspicious cases
    if result.get('match_score', 100) < 70:
        verdict, score = quick_cross_check(row, selected_model)
        validation_df.at[idx, 'cross_check_verdict'] = verdict
        validation_df.at[idx, 'cross_check_score'] = score

    # Rate limiting
    time.sleep(1)  # Adjust based on your API tier

    # Progress update every 10 items
    if (validated_count + error_count) % 10 == 0:
        print(f"Progress: {validated_count + error_count}/{len(validation_df)} | Mismatches found: {mismatch_count}")

print(f"\n✓ Validation complete!")
print(f"  Successfully validated: {validated_count}")
print(f"  Errors: {error_count}")
print(f"  Potential mismatches: {mismatch_count}")

STARTING VALIDATION PROCESS
Validating 100 studies...
Estimated time: 300 seconds
Estimated cost: ~$0.10 (varies by model)


Validating:   0%|          | 0/100 [00:00<?, ?it/s]/tmp/ipython-input-2189024698.py:8: DeprecationWarning: Call to deprecated function create.
  response = together.Complete.create(
/usr/local/lib/python3.12/dist-packages/together/legacy/complete.py:23: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment variable instead.
  warnings.warn(API_KEY_WARNING)
/tmp/ipython-input-2189024698.py:64: DeprecationWarning: Call to deprecated function create.
  response = together.Complete.create(
Validating:  10%|█         | 10/100 [01:20<11:43,  7.82s/it]

Progress: 10/100 | Mismatches found: 0


Validating:  20%|██        | 20/100 [03:11<14:33, 10.92s/it]

Progress: 20/100 | Mismatches found: 0


...

Progress: 80/100 | Mismatches found: 0


Validating:  90%|█████████ | 90/100 [14:15<01:28,  8.87s/it]

Progress: 90/100 | Mismatches found: 0


Validating: 100%|██████████| 100/100 [15:47<00:00,  9.48s/it]

Progress: 100/100 | Mismatches found: 0

✓ Validation complete!
  Successfully validated: 0
  Errors: 100
  Potential mismatches: 0


In [ ]:
print("=" * 80)
print("VALIDATION RESULTS ANALYSIS")
print("=" * 80)

# Overall statistics
valid_scores = validation_df[validation_df['validation_score'] > -1]['validation_score']
print("\nMatch Score Statistics:")
print(f"  Mean score: {valid_scores.mean():.1f}")
print(f"  Median score: {valid_scores.median():.1f}")
print(f"  Min score: {valid_scores.min():.1f}")
print(f"  Max score: {valid_scores.max():.1f}")

# Verdict distribution
print("\nVerdict Distribution:")
verdict_counts = validation_df['validation_verdict'].value_counts()
for verdict, count in verdict_counts.items():
    pct = count / len(validation_df) * 100
    print(f"  {verdict:20s}: {count:3d} ({pct:5.1f}%)")

# Confidence distribution
print("\nConfidence Levels:")
confidence_counts = validation_df['validation_confidence'].value_counts()
for conf, count in confidence_counts.items():
    pct = count / len(validation_df) * 100
    print(f"  {conf:10s}: {count:3d} ({pct:5.1f}%)")

# Identify problematic studies
problematic = validation_df[
    validation_df['validation_verdict'].isin(['likely_mismatch', 'definite_mismatch', 'uncertain'])
].copy()

print(f"\n⚠️ Found {len(problematic)} potentially problematic matches")

if len(problematic) > 0:
    print("\nSample problematic studies:")
    display_cols = ['rct_id', 'Title', 'validation_score', 'validation_verdict', 'validation_issues']
    print(problematic[display_cols].head(10).to_string())

VALIDATION RESULTS ANALYSIS

Match Score Statistics:
  Mean score: nan
  Median score: nan
  Min score: nan
  Max score: nan

Verdict Distribution:
  error               : 100 (100.0%)

Confidence Levels:

⚠️ Found 0 potentially problematic matches


In [ ]:
import pandas as pd
import numpy as np
import json
import time
from typing import Dict, List, Tuple
import os
from datetime import datetime
from tqdm import tqdm

# Install together package
!pip install --upgrade together --quiet

# Import Together with new API
from together import Together

# Set up TogetherAI with new API format
print("Setting up TogetherAI...")

# Method 1: Direct API key
# client = Together(api_key="YOUR_API_KEY_HERE")

# Method 2: Environment variable (recommended)
import getpass
api_key = getpass.getpass('Enter your TogetherAI API key: ')
os.environ['TOGETHER_API_KEY'] = api_key
client = Together(api_key=api_key)

print("✓ TogetherAI configured with new API")

# Base path for data
base_path = '/content/drive/MyDrive/AEA_RCT_Parsed/'

Setting up TogetherAI...
Enter your TogetherAI API key: ··········
✓ TogetherAI configured with new API


In [ ]:
print("Testing API connection...")

# Test with a simple prompt
test_prompt = "Say 'API working' if you receive this message."

try:
    # New API format for chat completions
    response = client.chat.completions.create(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": test_prompt}
        ],
        temperature=0.1,
        max_tokens=50
    )

    test_result = response.choices[0].message.content
    print(f"✅ API Test Successful! Response: {test_result}")

except Exception as e:
    print(f"❌ API Test Failed!")
    print(f"Error Type: {type(e).__name__}")
    print(f"Error Message: {str(e)}")
    print("\nTroubleshooting:")
    print("1. Check if your API key is valid")
    print("2. Verify you have credits in your TogetherAI account")
    print("3. Check if the model name is correct")
    print("4. Try a different model from the list below")

Testing API connection...
✅ API Test Successful! Response:  API working! I'm here to help you with any questions or tasks you have. Let me know how I can assist you today.


In [ ]:
print("\nLoading consolidated dataset...")
try:
    df = pd.read_csv(f'{base_path}consolidated_rct_dataset_clean.csv')
    print(f"✓ Loaded {len(df)} studies")

    # Show data sample
    print("\nDataset columns:", df.columns.tolist())
    print("\nFirst study sample:")
    print(f"Title: {df.iloc[0]['Title'][:100] if 'Title' in df.columns else 'N/A'}")
    print(f"Abstract length: {len(str(df.iloc[0]['Abstract'])) if 'Abstract' in df.columns else 'N/A'}")
    print(f"Findings length: {len(str(df.iloc[0]['Findings'])) if 'Findings' in df.columns else 'N/A'}")

except Exception as e:
    print(f"❌ Error loading data: {e}")
    print("Please check the file path")

# Create small test sample first
test_size = 5  # Start with just 5 for debugging
validation_df = df.sample(n=test_size, random_state=42).copy()
print(f"\n✓ Selected {len(validation_df)} studies for initial testing")


Loading consolidated dataset...
✓ Loaded 2281 studies

Dataset columns: ['Title', 'Country', 'Subregion', 'Abstract', 'Description_of_intervention', 'Primary_outcomes', 'Secondary_outcomes', 'Population', 'Search_vector', 'keywords', 'keywords_additional', 'jel_codes', 'keywords_sector', 'keywords_mechanisms', 'keywords_implementation', 'keywords_context', 'Researchers', 'Researcher_affiliation', 'Year', 'Findings', 'rct_id']

First study sample:
Title: Two Approaches to Community Development
Abstract length: 1243
Findings length: 377

✓ Selected 5 studies for initial testing


In [ ]:
def create_simple_validation_prompt(row: pd.Series) -> str:
    """Create a simpler validation prompt that's more likely to work."""

    # Keep it short to avoid token limits
    title = str(row.get('Title', 'N/A'))[:200]
    abstract = str(row.get('Abstract', 'N/A'))[:300]
    findings = str(row.get('Findings', 'N/A'))[:400]

    prompt = f"""Check if these findings match this study.

Study Title: {title}

Study Abstract: {abstract}

Findings: {findings}

Question: Do the findings above appear to be from the same study described in the title and abstract?

Answer with a number 0-100 (100 means definitely same study, 0 means definitely different study) and one word verdict.
Format: [SCORE] [VERDICT]
Example: 85 likely_match

Your answer:"""

    return prompt

def validate_with_new_api(row: pd.Series, model: str = "mistralai/Mixtral-8x7B-Instruct-v0.1") -> Dict:
    """Validate using the new Together API format."""

    prompt = create_simple_validation_prompt(row)

    try:
        # New API format
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a research validation expert. Be concise."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
            max_tokens=50
        )

        result_text = response.choices[0].message.content.strip()

        # Parse simple response format
        parts = result_text.split()
        if len(parts) >= 2:
            try:
                score = float(parts[0])
                verdict = parts[1]

                # Categorize verdict
                if score >= 80:
                    verdict_category = "correct_match"
                elif score >= 60:
                    verdict_category = "likely_match"
                elif score >= 40:
                    verdict_category = "uncertain"
                elif score >= 20:
                    verdict_category = "likely_mismatch"
                else:
                    verdict_category = "definite_mismatch"

                return {
                    'success': True,
                    'match_score': score,
                    'verdict': verdict_category,
                    'raw_response': result_text,
                    'confidence': 'medium'
                }
            except ValueError:
                pass

        # If parsing failed, return error
        return {
            'success': False,
            'error': 'Failed to parse response',
            'raw_response': result_text,
            'match_score': -1,
            'verdict': 'parse_error'
        }

    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'error_type': type(e).__name__,
            'match_score': -1,
            'verdict': 'api_error'
        }

In [ ]:
print("=" * 80)
print("DEBUG TEST RUN")
print("=" * 80)

# Test on first few studies with detailed error reporting
for i, (idx, row) in enumerate(validation_df.head(3).iterrows()):
    print(f"\n--- Testing Study {i+1} ---")
    print(f"RCT ID: {row['rct_id']}")
    print(f"Title: {str(row['Title'])[:100]}...")

    result = validate_with_new_api(row)

    if result['success']:
        print(f"✅ Success!")
        print(f"   Score: {result['match_score']}")
        print(f"   Verdict: {result['verdict']}")
        print(f"   Raw response: {result['raw_response']}")
    else:
        print(f"❌ Failed!")
        print(f"   Error type: {result.get('error_type', 'Unknown')}")
        print(f"   Error: {result.get('error', 'Unknown error')}")
        print(f"   Raw response: {result.get('raw_response', 'No response')}")

    time.sleep(2)  # Rate limiting

DEBUG TEST RUN

--- Testing Study 1 ---
RCT ID: 3939.0
Title: Salience of the energy-efficiency trade-off and purchase of energy efficient appliances...
✅ Success!
   Score: 90.0
   Verdict: correct_match
   Raw response: 90 mismatch

The findings do not match the study title and abstract. The study is about purchasing more energy-efficient appliances when energy cost information is made salient, while the findings indicate purchasing less energy-efficient appliances.

--- Testing Study 2 ---
RCT ID: 2631.0
Title: Evaluation of Big Word Club...
✅ Success!
   Score: 90.0
   Verdict: correct_match
   Raw response: 90 match. The findings mention a specific program ("Big Word Club") and its effectiveness in teaching vocabulary, which aligns with the study title and abstract. The study's focus on evaluating vocabulary programs and the use of a

--- Testing Study 3 ---
RCT ID: 692.0
Title: Nudging and Intrapreneurship...
✅ Success!
   Score: 90.0
   Verdict: correct_match
   Raw response: 90

In [ ]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from tqdm import tqdm
import os

# Install together package
!pip install --upgrade together --quiet

# Import Together with new API
from together import Together

# Set up TogetherAI
print("Setting up TogetherAI...")
import getpass
api_key = getpass.getpass('Enter your TogetherAI API key: ')
os.environ['TOGETHER_API_KEY'] = api_key
client = Together(api_key=api_key)
print("✓ TogetherAI configured")

# Base path for data
base_path = '/content/drive/MyDrive/AEA_RCT_Parsed/'

Setting up TogetherAI...
Enter your TogetherAI API key: ··········
✓ TogetherAI configured


In [ ]:
print("=" * 80)
print("LOADING DATA AND SELECTING RANDOM SAMPLE")
print("=" * 80)

# Load full dataset
df = pd.read_csv(f'{base_path}consolidated_rct_dataset_clean.csv')
print(f"✓ Loaded {len(df)} total studies from dataset")

# Select 40 random studies
SAMPLE_SIZE = 40
validation_df = df.sample(n=SAMPLE_SIZE, random_state=42).copy()
print(f"✓ Randomly selected {SAMPLE_SIZE} studies for validation")

# Show the RCT IDs selected
print("\nRCT IDs selected for validation:")
selected_ids = validation_df['rct_id'].tolist()
print(f"{selected_ids[:10]} ... (showing first 10 of {len(selected_ids)})")

LOADING DATA AND SELECTING RANDOM SAMPLE
✓ Loaded 2281 total studies from dataset
✓ Randomly selected 40 studies for validation

RCT IDs selected for validation:
[3939.0, 2631.0, 692.0, 2964.0, 502.0, 7252.0, 8635.0, 9791.0, 370.0, 1446.0] ... (showing first 10 of 40)


In [ ]:
print("\n" + "=" * 80)
print("TESTING API CONNECTION")
print("=" * 80)

try:
    response = client.chat.completions.create(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        messages=[{"role": "user", "content": "Say 'OK' if working"}],
        max_tokens=10,
        temperature=0.1
    )
    print(f"✅ API Test Successful! Response: {response.choices[0].message.content}")
    model_to_use = "mistralai/Mixtral-8x7B-Instruct-v0.1"

except Exception as e:
    print(f"❌ Mixtral failed, trying Mistral-7B...")
    try:
        response = client.chat.completions.create(
            model="mistralai/Mistral-7B-Instruct-v0.1",
            messages=[{"role": "user", "content": "Say 'OK' if working"}],
            max_tokens=10,
            temperature=0.1
        )
        print(f"✅ Mistral-7B works!")
        model_to_use = "mistralai/Mistral-7B-Instruct-v0.1"
    except:
        print("❌ API connection failed. Please check your key.")
        model_to_use = None

print(f"\nUsing model: {model_to_use}")


TESTING API CONNECTION
✅ API Test Successful! Response:  OK, I'm here and ready to assist

Using model: mistralai/Mixtral-8x7B-Instruct-v0.1


In [ ]:
def validate_study(row: pd.Series, model: str) -> dict:
    """Validate if findings match the study."""

    # Extract key fields (truncate to avoid token limits)
    title = str(row.get('Title', 'N/A'))[:300]
    abstract = str(row.get('Abstract', 'N/A'))[:400]
    intervention = str(row.get('Description_of_intervention', 'N/A'))[:200]
    outcomes = str(row.get('Primary_outcomes', 'N/A'))[:200]
    findings = str(row.get('Findings', 'N/A'))[:500]
    year = str(row.get('Year', 'N/A'))
    country = str(row.get('Country', 'N/A'))

    prompt = f"""Analyze if these research findings correctly match the study described.

STUDY INFORMATION:
Title: {title}
Year: {year}
Country: {country}
Abstract: {abstract}
Intervention: {intervention}
Primary Outcomes: {outcomes}

FINDINGS TO VALIDATE:
{findings}

QUESTION: Do these findings appear to be from the same study described above?

Provide:
1. A match score from 0-100 (100 = definitely same study, 0 = definitely different)
2. Main reason for your score
3. Any red flags noticed

Format your response as:
SCORE: [number]
REASON: [one sentence explanation]
FLAGS: [any issues, or "none"]

Response:"""

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a research validation expert. Be direct and concise."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1,
            max_tokens=150
        )

        result_text = response.choices[0].message.content.strip()

        # Parse the response
        score = -1
        reason = ""
        flags = ""

        for line in result_text.split('\n'):
            if 'SCORE:' in line:
                try:
                    score = float(line.split('SCORE:')[1].strip().split()[0])
                except:
                    score = -1
            elif 'REASON:' in line:
                reason = line.split('REASON:')[1].strip()
            elif 'FLAGS:' in line or 'FLAG:' in line:
                flags = line.split(':', 1)[1].strip() if ':' in line else ""

        # Determine verdict based on score
        if score >= 80:
            verdict = "✅ MATCH"
        elif score >= 60:
            verdict = "🟡 LIKELY MATCH"
        elif score >= 40:
            verdict = "🟠 UNCERTAIN"
        elif score >= 20:
            verdict = "🔴 LIKELY MISMATCH"
        elif score >= 0:
            verdict = "❌ MISMATCH"
        else:
            verdict = "❓ ERROR"

        return {
            'success': True,
            'score': score,
            'verdict': verdict,
            'reason': reason,
            'flags': flags,
            'raw_response': result_text
        }

    except Exception as e:
        return {
            'success': False,
            'score': -1,
            'verdict': "❓ ERROR",
            'reason': str(e)[:100],
            'flags': 'API error',
            'raw_response': ''
        }

In [ ]:
print("\n" + "=" * 80)
print(f"STARTING VALIDATION OF {SAMPLE_SIZE} RANDOM STUDIES")
print("=" * 80)
print(f"Estimated time: {SAMPLE_SIZE * 2} seconds")
print(f"Starting at: {datetime.now().strftime('%H:%M:%S')}\n")

# Initialize results storage
results = []
success_count = 0
error_count = 0
mismatch_count = 0
uncertain_count = 0

# Process each study
for i, (idx, row) in enumerate(validation_df.iterrows(), 1):
    print(f"\n--- Study {i}/{SAMPLE_SIZE} ---")
    print(f"RCT ID: {row['rct_id']}")
    print(f"Title: {str(row['Title'])[:80]}...")

    # Validate
    result = validate_study(row, model_to_use)

    # Store result
    results.append({
        'rct_id': row['rct_id'],
        'title': row['Title'],
        'score': result['score'],
        'verdict': result['verdict'],
        'reason': result['reason'],
        'flags': result['flags']
    })

    # Print result
    print(f"Score: {result['score']}")
    print(f"Verdict: {result['verdict']}")
    print(f"Reason: {result['reason']}")
    if result['flags'] and result['flags'] != 'none':
        print(f"⚠️ Flags: {result['flags']}")

    # Update counters
    if result['success']:
        success_count += 1
        if result['score'] < 40:
            mismatch_count += 1
        elif result['score'] < 60:
            uncertain_count += 1
    else:
        error_count += 1

    # Progress indicator every 10
    if i % 10 == 0:
        print(f"\n{'='*50}")
        print(f"PROGRESS: {i}/{SAMPLE_SIZE} completed")
        print(f"Success: {success_count} | Errors: {error_count} | Mismatches: {mismatch_count}")
        print(f"{'='*50}")

    # Rate limiting
    time.sleep(1.5)

print(f"\n✅ VALIDATION COMPLETE at {datetime.now().strftime('%H:%M:%S')}")


STARTING VALIDATION OF 40 RANDOM STUDIES
Estimated time: 80 seconds
Starting at: 01:57:21


--- Study 1/40 ---
RCT ID: 3939.0
Title: Salience of the energy-efficiency trade-off and purchase of energy efficient app...
Score: 0.0
Verdict: ❌ MISMATCH
Reason: The finding states that more salient energy cost information leads to purchasing less energy-efficient appliances, while the study description implies that it should lead to purchasing more energy-efficient appliances.
⚠️ Flags: None. The discrepancy is clear and direct.

--- Study 2/40 ---
RCT ID: 2631.0
Title: Evaluation of Big Word Club...
Score: 90.0
Verdict: ✅ MATCH
Reason: The findings describe an evaluation of a digital learning program, Big Word Club, which matches the study description. The primary outcome of improved receptive vocabulary after four months of use is also consistent with the findings.
⚠️ Flags: The only discrepancy is the lack of information on the standardized vocabulary assessment (PPVT) mentioned in the fi

In [ ]:
print("\n" + "=" * 80)
print("COMPLETE VALIDATION RESULTS")
print("=" * 80)

# Convert results to DataFrame for analysis
results_df = pd.DataFrame(results)

# Overall statistics
print(f"\n📊 OVERALL STATISTICS:")
print(f"Total Studies Validated: {SAMPLE_SIZE}")
print(f"Successful Validations: {success_count}")
print(f"Failed Validations: {error_count}")

# Score distribution
valid_scores = results_df[results_df['score'] >= 0]['score']
if len(valid_scores) > 0:
    print(f"\n📈 SCORE DISTRIBUTION:")
    print(f"Mean Score: {valid_scores.mean():.1f}")
    print(f"Median Score: {valid_scores.median():.1f}")
    print(f"Min Score: {valid_scores.min():.1f}")
    print(f"Max Score: {valid_scores.max():.1f}")
    print(f"Std Dev: {valid_scores.std():.1f}")

# Verdict counts
print(f"\n🎯 VERDICT DISTRIBUTION:")
verdict_counts = results_df['verdict'].value_counts()
for verdict, count in verdict_counts.items():
    percentage = (count / SAMPLE_SIZE) * 100
    print(f"{verdict}: {count} ({percentage:.1f}%)")

# Quality assessment
print(f"\n📋 QUALITY ASSESSMENT:")
high_quality = len(results_df[results_df['score'] >= 80])
medium_quality = len(results_df[(results_df['score'] >= 60) & (results_df['score'] < 80)])
low_quality = len(results_df[(results_df['score'] >= 0) & (results_df['score'] < 60)])

print(f"High Quality Matches (80+): {high_quality} ({high_quality/SAMPLE_SIZE*100:.1f}%)")
print(f"Medium Quality Matches (60-79): {medium_quality} ({medium_quality/SAMPLE_SIZE*100:.1f}%)")
print(f"Low Quality/Mismatches (<60): {low_quality} ({low_quality/SAMPLE_SIZE*100:.1f}%)")

# Data quality grade
if valid_scores.mean() >= 85:
    grade = "A - Excellent"
elif valid_scores.mean() >= 75:
    grade = "B - Good"
elif valid_scores.mean() >= 65:
    grade = "C - Acceptable"
elif valid_scores.mean() >= 55:
    grade = "D - Needs Review"
else:
    grade = "F - Major Issues"

print(f"\n🏆 OVERALL DATA QUALITY GRADE: {grade}")


COMPLETE VALIDATION RESULTS

📊 OVERALL STATISTICS:
Total Studies Validated: 40
Successful Validations: 40
Failed Validations: 0

📈 SCORE DISTRIBUTION:
Mean Score: 71.0
Median Score: 80.0
Min Score: 0.0
Max Score: 100.0
Std Dev: 29.0

🎯 VERDICT DISTRIBUTION:
✅ MATCH: 30 (75.0%)
🔴 LIKELY MISMATCH: 4 (10.0%)
❌ MISMATCH: 3 (7.5%)
🟡 LIKELY MATCH: 3 (7.5%)

📋 QUALITY ASSESSMENT:
High Quality Matches (80+): 30 (75.0%)
Medium Quality Matches (60-79): 3 (7.5%)
Low Quality/Mismatches (<60): 7 (17.5%)

🏆 OVERALL DATA QUALITY GRADE: C - Acceptable


In [ ]:
print("\n" + "=" * 80)
print("PROBLEMATIC STUDIES REQUIRING REVIEW")
print("=" * 80)

# Filter for problematic studies
problematic = results_df[results_df['score'] < 60].sort_values('score')

if len(problematic) > 0:
    print(f"\n⚠️ Found {len(problematic)} studies with potential issues (score < 60):\n")

    for idx, row in problematic.iterrows():
        print(f"{'='*60}")
        print(f"RCT ID: {row['rct_id']}")
        # Print full title, wrapped if needed
        print(f"Title: {row['title']}")
        print(f"Score: {row['score']:.1f}")
        print(f"Verdict: {row['verdict']}")
        print(f"Reason: {row['reason']}")
        if row['flags'] and row['flags'] != 'none':
            print(f"Flags: {row['flags']}")
else:
    print("\n✅ No problematic studies found! All scores >= 60")

# Also show uncertain ones with full titles
uncertain = results_df[(results_df['score'] >= 40) & (results_df['score'] < 60)]
if len(uncertain) > 0:
    print(f"\n🟠 UNCERTAIN STUDIES (40-59 score range):")
    print(f"Found {len(uncertain)} studies that may need manual review:\n")

    for idx, row in uncertain.iterrows():
        print(f"- RCT {row['rct_id']}: {row['title']}")
        print(f"  Score: {row['score']:.1f} | Reason: {row['reason']}")


PROBLEMATIC STUDIES REQUIRING REVIEW

⚠️ Found 7 studies with potential issues (score < 60):

RCT ID: 3939.0
Title: Salience of the energy-efficiency trade-off and purchase of energy efficient appliances
Score: 0.0
Verdict: ❌ MISMATCH
Reason: The finding states that more salient energy cost information leads to purchasing less energy-efficient appliances, while the study description implies that it should lead to purchasing more energy-efficient appliances.
Flags: None. The discrepancy is clear and direct.
RCT ID: 1446.0
Title: From natural variation to optimal policy? The importance of endogenous peer group formation.
Score: 0.0
Verdict: ❌ MISMATCH
Reason: The study findings describe a negative and significant treatment effect for the students the intervention aimed to help, but the study description explains that the intervention was designed to maximize the academic performance of the lowest ability students. These outcomes are contradictory.
Flags: The findings mention "students w

In [ ]:
print("\n" + "=" * 80)
print("COMPLETE VALIDATION RESULTS")
print("=" * 80)

# Convert results to DataFrame for analysis
results_df = pd.DataFrame(results)

# Overall statistics
print(f"\n📊 OVERALL STATISTICS:")
print(f"Total Studies Validated: {SAMPLE_SIZE}")
print(f"Successful Validations: {success_count}")
print(f"Failed Validations: {error_count}")

# Score distribution
valid_scores = results_df[results_df['score'] >= 0]['score']
if len(valid_scores) > 0:
    print(f"\n📈 SCORE DISTRIBUTION:")
    print(f"Mean Score: {valid_scores.mean():.1f}")
    print(f"Median Score: {valid_scores.median():.1f}")
    print(f"Min Score: {valid_scores.min():.1f}")
    print(f"Max Score: {valid_scores.max():.1f}")
    print(f"Std Dev: {valid_scores.std():.1f}")

# Verdict counts
print(f"\n🎯 VERDICT DISTRIBUTION:")
verdict_counts = results_df['verdict'].value_counts()
for verdict, count in verdict_counts.items():
    percentage = (count / SAMPLE_SIZE) * 100
    print(f"{verdict}: {count} ({percentage:.1f}%)")

# Quality assessment
print(f"\n📋 QUALITY ASSESSMENT:")
high_quality = len(results_df[results_df['score'] >= 80])
medium_quality = len(results_df[(results_df['score'] >= 60) & (results_df['score'] < 80)])
low_quality = len(results_df[(results_df['score'] >= 0) & (results_df['score'] < 60)])

print(f"High Quality Matches (80+): {high_quality} ({high_quality/SAMPLE_SIZE*100:.1f}%)")
print(f"Medium Quality Matches (60-79): {medium_quality} ({medium_quality/SAMPLE_SIZE*100:.1f}%)")
print(f"Low Quality/Mismatches (<60): {low_quality} ({low_quality/SAMPLE_SIZE*100:.1f}%)")

# Data quality grade
if valid_scores.mean() >= 85:
    grade = "A - Excellent"
elif valid_scores.mean() >= 75:
    grade = "B - Good"
elif valid_scores.mean() >= 65:
    grade = "C - Acceptable"
elif valid_scores.mean() >= 55:
    grade = "D - Needs Review"
else:
    grade = "F - Major Issues"

print(f"\n🏆 OVERALL DATA QUALITY GRADE: {grade}")


COMPLETE VALIDATION RESULTS

📊 OVERALL STATISTICS:
Total Studies Validated: 40
Successful Validations: 40
Failed Validations: 0

📈 SCORE DISTRIBUTION:
Mean Score: 71.0
Median Score: 80.0
Min Score: 0.0
Max Score: 100.0
Std Dev: 29.0

🎯 VERDICT DISTRIBUTION:
✅ MATCH: 30 (75.0%)
🔴 LIKELY MISMATCH: 4 (10.0%)
❌ MISMATCH: 3 (7.5%)
🟡 LIKELY MATCH: 3 (7.5%)

📋 QUALITY ASSESSMENT:
High Quality Matches (80+): 30 (75.0%)
Medium Quality Matches (60-79): 3 (7.5%)
Low Quality/Mismatches (<60): 7 (17.5%)

🏆 OVERALL DATA QUALITY GRADE: C - Acceptable


In [ ]:
import pandas as pd
import textwrap

# Base path
base_path = '/content/drive/MyDrive/AEA_RCT_Parsed/'

# Load the full dataset
print("Loading consolidated dataset...")
df = pd.read_csv(f'{base_path}consolidated_rct_dataset_clean.csv')

# Recreate the same 40-study sample using the same random seed
SAMPLE_SIZE = 40
validation_df = df.sample(n=SAMPLE_SIZE, random_state=42).copy()
print(f"✓ Loaded validation sample of {len(validation_df)} studies")

# Define the suspicious RCT IDs to inspect
suspicious_ids = [15014, 9791, 728]
print(f"\n🔍 Inspecting RCT IDs: {suspicious_ids}")

Loading consolidated dataset...
✓ Loaded validation sample of 40 studies

🔍 Inspecting RCT IDs: [15014, 9791, 728]


In [ ]:
def print_wrapped(label, text, width=100, indent=4):
    """Print text with nice wrapping and indentation."""
    if pd.isna(text) or text == "":
        print(f"{label}: [Empty]")
    else:
        print(f"{label}:")
        wrapped = textwrap.fill(str(text), width=width, initial_indent=' '*indent,
                               subsequent_indent=' '*indent)
        print(wrapped)

def print_study_details(row):
    """Print all details of a study in a readable format."""
    print("\n" + "="*100)
    print(f"RCT ID: {row['rct_id']}")
    print("="*100)

    # Basic Information
    print("\n📚 BASIC INFORMATION:")
    print(f"Title: {row['Title']}")
    print(f"Year: {row['Year']}")
    print(f"Country: {row['Country']}")
    print(f"Subregion: {row.get('Subregion', 'N/A')}")

    # Researchers
    print("\n👥 RESEARCHERS:")
    print_wrapped("Researchers", row.get('Researchers', 'N/A'), width=95)
    print_wrapped("Affiliations", row.get('Researcher_affiliation', 'N/A'), width=95)

    # Study Design
    print("\n🔬 STUDY DESIGN:")
    print_wrapped("Abstract", row.get('Abstract', 'N/A'), width=95)
    print()
    print_wrapped("Description of Intervention", row.get('Description_of_intervention', 'N/A'), width=95)
    print()
    print_wrapped("Primary Outcomes", row.get('Primary_outcomes', 'N/A'), width=95)
    print()
    print_wrapped("Secondary Outcomes", row.get('Secondary_outcomes', 'N/A'), width=95)

    # Findings
    print("\n📊 FINDINGS:")
    print_wrapped("Findings", row.get('Findings', 'N/A'), width=95)

    # Classification
    print("\n🏷️ CLASSIFICATION:")
    print(f"Keywords: {row.get('keywords', 'N/A')}")
    print(f"Keywords Additional: {row.get('keywords_additional', 'N/A')}")
    print(f"JEL Codes: {row.get('jel_codes', 'N/A')}")
    print(f"Population: {row.get('Population', 'N/A')}")

    # Keywords subcategories
    print("\n📂 KEYWORD SUBCATEGORIES:")
    print(f"Sector: {row.get('keywords_sector', 'N/A')}")
    print(f"Mechanisms: {row.get('keywords_mechanisms', 'N/A')}")
    print(f"Implementation: {row.get('keywords_implementation', 'N/A')}")
    print(f"Context: {row.get('keywords_context', 'N/A')}")

In [ ]:
# Filter for suspicious studies
suspicious_studies = validation_df[validation_df['rct_id'].isin(suspicious_ids)]

if len(suspicious_studies) == 0:
    print(f"\n❌ None of the suspicious RCT IDs {suspicious_ids} are in the validation sample!")
    print("Checking if they exist in the full dataset...")

    # Check in full dataset
    suspicious_in_full = df[df['rct_id'].isin(suspicious_ids)]
    if len(suspicious_in_full) > 0:
        print(f"✓ Found {len(suspicious_in_full)} of these studies in the full dataset")
        print("Displaying them from the full dataset instead:\n")
        suspicious_studies = suspicious_in_full
    else:
        print("❌ These RCT IDs don't exist in the dataset")

# Display each study
for idx, row in suspicious_studies.iterrows():
    print_study_details(row)


RCT ID: 9791.0

📚 BASIC INFORMATION:
Title: Attitudes towards refugees and identity
Year: 2022
Country: Italy
Subregion: nan

👥 RESEARCHERS:
Researchers:
    Cristina Cattaneo; PI Name Daniela Grieco PI Affiliation CONTACT INVESTIGATOR PI Name Mario
    Macis PI Affiliation CONTACT INVESTIGATOR PI Name Nicola Lacetera PI Affiliation CONTACT
    INVESTIGATOR
Affiliations:
    RFF-CMCC and CMCC

🔬 STUDY DESIGN:
Abstract:
    Using data from an incentivised online survey with nationally representative Italians, we
    study natives’ attitudes towards refugees, some sharing a common European identity and some
    not sharing it, being from outside Europe. The project has three objectives. First, we aim
    to test if the way the asylum seekers are managed and relocated between the different
    European countries, affects the attitude towards refugees. To this aim, we prime a reform
    of the EU refugee legislative framework, which sets more equal allocation rules among
    member states

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary library
import pandas as pd

# Load the primary dataset
df = pd.read_csv('/content/drive/MyDrive/AEA_RCT_Parsed/structured_studies_full_v2.csv')

# Define the AI-generated keyword columns
ai_keyword_columns = ['keywords_sector', 'keywords_mechanisms',
                      'keywords_implementation', 'keywords_context']

# Create a boolean mask for studies with AI-generated keywords
# A study has AI keywords if at least one of the AI keyword columns is not null/empty
has_ai_keywords_mask = df[ai_keyword_columns].notna().any(axis=1)

# For string columns, also check they're not empty strings
for col in ai_keyword_columns:
    if col in df.columns and df[col].dtype == 'object':
        has_ai_keywords_mask = has_ai_keywords_mask & (df[col] != '')

# Split the dataset into two groups
studies_with_ai_keywords = df[has_ai_keywords_mask].copy()
studies_without_ai_keywords = df[~has_ai_keywords_mask].copy()

# Display counts for verification
print(f"Total studies: {len(df)}")
print(f"Studies WITH AI-generated keywords: {len(studies_with_ai_keywords)}")
print(f"Studies WITHOUT AI-generated keywords: {len(studies_without_ai_keywords)}")

# Save the two files
studies_with_ai_keywords.to_csv('/content/drive/MyDrive/AEA_RCT_Parsed/studies_with_ai_keywords.csv',
                                index=False)
studies_without_ai_keywords.to_csv('/content/drive/MyDrive/AEA_RCT_Parsed/studies_without_ai_keywords.csv',
                                   index=False)

print("\nFiles saved successfully:")
print("1. studies_with_ai_keywords.csv")
print("2. studies_without_ai_keywords.csv")

# Optional: Display sample data to verify the split
print("\n--- Sample of studies WITH AI keywords ---")
print(studies_with_ai_keywords[['rct_id', 'Title'] + ai_keyword_columns].head(3))

print("\n--- Sample of studies WITHOUT AI keywords ---")
print(studies_without_ai_keywords[['rct_id', 'Title'] + ai_keyword_columns].head(3))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total studies: 2289
Studies WITH AI-generated keywords: 2117
Studies WITHOUT AI-generated keywords: 172

Files saved successfully:
1. studies_with_ai_keywords.csv
2. studies_without_ai_keywords.csv

--- Sample of studies WITH AI keywords ---


KeyError: "['Title'] not in index"